The following is a datasheet containing the vehicle performace of a vehicle. The data is given out by an IoT placed in a vehicle. The data is in the form of hexcodes. 
https://bit.ly/3go6Dth

Below is a Pre-Processing code that processes the data to understand and work further on the data.

The output from the code: https://bit.ly/352hqnO

### **Data Processing.**

Importing Libraries.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
dataset = pd.read_csv('T_000356.CSV', 
                         names=['Time','Latitude','Longitude','Altitude','RPM' ,
                        'Driver Demand Torque (%)','Engine Load (%)','Engine Torque Mode','TPS (%)','Percent Load Curret Speed','Fuel Rate (L-Hr)',
                        'Vehicle Speed ','Inj Q Cur (mg-st)','Inj Q Tor (mg-st)','Boost Pressure (mBar)','Atmospheric Pressure (mBar)','Coolant Temperature (*C)',
                        'Oil Temperature (*C)','Boost Temperature (*C)','Oil Pressure (mBar)','Battery Voltage (V)','Cam Speed (rpm)',
                        'Rail Pressure (mBar)','Rail Pressure set (mBar)','MU PWM (%)','MU Vol (mm3-s)','Torque Rat','Torque (Nm)','TQ Limit Set','Main Injection (mg-st)',
                        'Pilot Injection (mg-st)','Pos 2 Injector (mg-st)',
                        'EGR Prop (%)','EGR Pos D (%)','EGR Pos A (%)','Clutch Switch','Brake Switch','Engine Grad (rpm)','param1','param2','0'])

In [ ]:
dataset = dataset.fillna(0)

Getting where the engine stops or when the file overflows.

In [ ]:
for i in range(0,len(dataset)):
  if dataset['Latitude'][i] == 'ENG STOPPED':
    mi = i
  elif dataset['Latitude'][i] == 'FILE OVERFLOW':
    mi = i

Collecting date and droping useless data.

In [ ]:
date_from_sheet = dataset.iloc[-9][0]
dataset.drop(dataset.tail(len(dataset)-mi).index,inplace = True)
dataset = dataset.drop(dataset.columns[-1],axis=1)

In [ ]:
dataset

,Time,Latitude,Longitude,Altitude,RPM,Driver Demand Torque (%),Engine Load (%),Engine Torque Mode,TPS (%),Percent Load Curret Speed,Fuel Rate (L-Hr),Vehicle Speed,Inj Q Cur (mg-st),Inj Q Tor (mg-st),Boost Pressure (mBar),Atmospheric Pressure (mBar),Coolant Temperature (*C),Oil Temperature (*C),Boost Temperature (*C),Oil Pressure (mBar),Battery Voltage (V),Cam Speed (rpm),Rail Pressure (mBar),Rail Pressure set (mBar),MU PWM (%),MU Vol (mm3-s),Torque Rat,Torque (Nm),TQ Limit Set,Main Injection (mg-st),Pilot Injection (mg-st),Pos 2 Injector (mg-st),EGR Prop (%),EGR Pos D (%),EGR Pos A (%),Clutch Switch,Brake Switch,Engine Grad (rpm),param1,param2
0,235945,0,0,0,5011,7d,85,60,00,0e,1600,0000,01df,01c4,0392,039c,0d30,0cd3,0cc6,1135,0552,044f,0c84,0cb2,081a,014a,04a9,0293,00aa,017b,006c,0.0,fc1f,f666,fd64,1.0,0.0,001b,000000ff,07ffffff
1,235946,0,0,0,3011,7d,85,60,00,0f,1700,0000,01ef,01d2,0391,039c,0d30,0cd3,0cbe,1127,0558,044c,0c99,0cb2,081c,013f,04ce,0296,00aa,0177,006c,0.0,fc28,f666,fd64,1.0,0.0,0000,000000ff,07ffffff
2,235947,0,0,0,3411,7d,85,50,00,0e,1600,0000,01e1,01da,0390,039c,0d2f,0cd3,0cb8,10f8,0569,044c,0caf,0cb2,0802,0149,04ea,029c,00aa,017c,006c,0.0,fc2f,f666,fd64,1.0,0.0,0000,000000ff,07ffffff
3,235948,0,0,0,3011,7d,85,60,00,0f,1700,0000,01e5,01de,0390,039c,0d2e,0cd4,0cb3,1113,0570,044c,0caf,0cb2,07ef,014d,04c1,0298,00aa,0179,006c,0.0,fc38,f666,fd64,1.0,0.0,0000,000001ff,07ffffff
4,235949,0,0,0,3011,7d,85,50,00,0e,1600,0000,01de,01df,038f,039b,0d2d,0cd4,0cae,10cf,0574,044d,0caf,0cb2,07f1,014e,04b2,0285,00aa,0164,006c,0.0,fc38,f666,fd64,1.0,0.0,ffe5,000000ff,07ffffff
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4757,174750,1835.8932,07343.6899,0588.6,0013,7d,84,30,00,0b,1300,0525,017a,00d8,03c1,03af,0e01,0e02,0c0e,0bf3,0562,04c5,0caf,0cb2,0847,017d,03b9,022c,00aa,0143,0064,0.0,fc2d,f666,fd7f,1.0,3.0,003c,000000ff,07ffffff
4758,174751,1835.8919,07343.6881,0588.6,c412,8e,96,21,00,0c,1300,04b7,0189,011d,03bb,03af,0e02,0e03,0c0f,0bd1,055f,04b2,0cc5,0cb2,0846,0185,0422,0231,00aa,0135,0064,0.0,fc2f,f666,fd7f,1.0,0.0,003c,000000ff,07ffffff
4759,174752,1835.8910,07343.6867,0588.6,3812,c2,b8,5c,19,32,4100,04d0,0512,0937,03b9,03af,0e02,0e02,0c12,0b44,0566,04ab,113d,1806,0430,07da,2000,0fff,08aa,083e,0064,0.0,fc30,f666,fd7f,1.0,0.0,001d,000000ff,07ffffff
4760,174753,1835.8902,07343.6854,0589.1,0c15,c4,ba,7c,8b,64,8200,056d,0a6c,0a14,03d2,03af,0e02,0e03,0c13,0bc4,0558,04a9,197d,18f5,0774,037d,1fd7,1157,08aa,097a,0064,0.0,fc2c,f666,fd7f,1.0,0.0,0166,000001ff,07ffffff


Date and Time

Date extraction

In [ ]:
date_from_sheet = date_from_sheet.split()
date_raw = []
date_raw.append(date_from_sheet[2])

for number in date_raw:
    str_number = str(number)
    date = ('{}/{}/20{}'. format(str_number[:2], str_number[2:4], str_number[4:]))

print('Date:',date)

Date: 04/05/2021


Time 

In [ ]:
raw_time_converted = []
for a in dataset['Time']:

  h = int(a[0:2])
  m = int(a[2:4])
  s = str(a[4:6])

  h = h+5
  m = m+30

  if m == 60:
    h = h+1
    m = str(00)

  elif m > 60:
    m = m - 60
    h = h + 1

  raw_time_converted.append((str(h).zfill(2))+(str(m).zfill(2))+(str(s).zfill(2)))

In [ ]:
raw_time_new = []

for each_time in raw_time_converted:
    str_time = str(each_time)
    time = ('{}:{}:{}'. format(str_time[:2], str_time[2:4], str_time[4:]))
    raw_time_new.append(time)

Combining date time column

In [ ]:
date_time_list = []

for time_new in raw_time_new:
  date_time_list.append(date+' '+time_new)

In [ ]:
dataset.drop('Time',axis=1)
dataset['Time'] = (date_time_list)

In [ ]:
dataset.head()

,Time,Latitude,Longitude,Altitude,RPM,Driver Demand Torque (%),Engine Load (%),Engine Torque Mode,TPS (%),Percent Load Curret Speed,Fuel Rate (L-Hr),Vehicle Speed,Inj Q Cur (mg-st),Inj Q Tor (mg-st),Boost Pressure (mBar),Atmospheric Pressure (mBar),Coolant Temperature (*C),Oil Temperature (*C),Boost Temperature (*C),Oil Pressure (mBar),Battery Voltage (V),Cam Speed (rpm),Rail Pressure (mBar),Rail Pressure set (mBar),MU PWM (%),MU Vol (mm3-s),Torque Rat,Torque (Nm),TQ Limit Set,Main Injection (mg-st),Pilot Injection (mg-st),Pos 2 Injector (mg-st),EGR Prop (%),EGR Pos D (%),EGR Pos A (%),Clutch Switch,Brake Switch,Engine Grad (rpm),param1,param2
0,04/05/2021 29:29:45,0,0,0,5011,7d,85,60,00,0e,1600,0000,01df,01c4,0392,039c,0d30,0cd3,0cc6,1135,0552,044f,0c84,0cb2,081a,014a,04a9,0293,00aa,017b,006c,0.0,fc1f,f666,fd64,1.0,0.0,001b,000000ff,07ffffff
1,04/05/2021 29:29:46,0,0,0,3011,7d,85,60,00,0f,1700,0000,01ef,01d2,0391,039c,0d30,0cd3,0cbe,1127,0558,044c,0c99,0cb2,081c,013f,04ce,0296,00aa,0177,006c,0.0,fc28,f666,fd64,1.0,0.0,0000,000000ff,07ffffff
2,04/05/2021 29:29:47,0,0,0,3411,7d,85,50,00,0e,1600,0000,01e1,01da,0390,039c,0d2f,0cd3,0cb8,10f8,0569,044c,0caf,0cb2,0802,0149,04ea,029c,00aa,017c,006c,0.0,fc2f,f666,fd64,1.0,0.0,0000,000000ff,07ffffff
3,04/05/2021 29:29:48,0,0,0,3011,7d,85,60,00,0f,1700,0000,01e5,01de,0390,039c,0d2e,0cd4,0cb3,1113,0570,044c,0caf,0cb2,07ef,014d,04c1,0298,00aa,0179,006c,0.0,fc38,f666,fd64,1.0,0.0,0000,000001ff,07ffffff
4,04/05/2021 29:29:49,0,0,0,3011,7d,85,50,00,0e,1600,0000,01de,01df,038f,039b,0d2d,0cd4,0cae,10cf,0574,044d,0caf,0cb2,07f1,014e,04b2,0285,00aa,0164,006c,0.0,fc38,f666,fd64,1.0,0.0,ffe5,000000ff,07ffffff


Latitude

In [ ]:
def dm(x):
    degrees = int(x) // 100
    minutes = x - 100*degrees

    return degrees, minutes

def decimal_degrees(degrees, minutes):
    return degrees + minutes/60 

lat_list = []
for i in dataset['Latitude']:
  if i != 0:
    lat_list.append(decimal_degrees(*dm(float(i))))
  elif i == 0:
    lat_list.append(0)

long_list = []
for i in dataset['Longitude']:
  if i != 0:
    long_list.append(decimal_degrees(*dm(float(i))))
  elif i == 0:
    long_list.append(0)

In [ ]:
dataset.drop('Latitude',axis=1)
dataset['Latitude'] = (lat_list)

dataset.drop('Longitude',axis=1)
dataset['Longitude'] = (long_list)

In [ ]:
dataset.iloc[50:55]

,Time,Latitude,Longitude,Altitude,RPM,Driver Demand Torque (%),Engine Load (%),Engine Torque Mode,TPS (%),Percent Load Curret Speed,Fuel Rate (L-Hr),Vehicle Speed,Inj Q Cur (mg-st),Inj Q Tor (mg-st),Boost Pressure (mBar),Atmospheric Pressure (mBar),Coolant Temperature (*C),Oil Temperature (*C),Boost Temperature (*C),Oil Pressure (mBar),Battery Voltage (V),Cam Speed (rpm),Rail Pressure (mBar),Rail Pressure set (mBar),MU PWM (%),MU Vol (mm3-s),Torque Rat,Torque (Nm),TQ Limit Set,Main Injection (mg-st),Pilot Injection (mg-st),Pos 2 Injector (mg-st),EGR Prop (%),EGR Pos D (%),EGR Pos A (%),Clutch Switch,Brake Switch,Engine Grad (rpm),param1,param2
50,04/05/2021 21:59:24,18.900668,73.909515,0217.4,b812,7d,85,40,00,0f,1800,0e94,01d7,0135,039b,039c,0d2a,0ce8,0c49,10cf,0575,04b2,0bd3,0cb2,07f3,0176,04bc,02a0,00aa,0171,006d,0.0,fc35,f666,fd7f,1.0,0.0,ffc5,000001ff,07ffffff
51,04/05/2021 21:59:25,18.900592,73.909465,0217.4,c012,7d,85,00,00,0e,1800,0e99,01bc,0197,0395,039c,0d2c,0ce9,0c4b,10bb,056d,04b0,0cdb,0cb2,07f6,0162,04a1,0293,00aa,0170,006c,0.0,fc35,f666,fd7f,1.0,0.0,003c,000000ff,07ffffff
52,04/05/2021 21:59:25,18.900592,73.909465,0217.4,d012,7d,85,00,00,0d,1800,0e99,01bc,0197,0395,039c,0d2d,0cea,0c4c,10ae,056c,04b0,0c99,0cb2,07f5,016e,0499,0283,00aa,016a,006c,0.0,fc35,f666,fd7f,1.0,0.0,001e,0000007f,07ffffff
53,04/05/2021 21:59:27,18.900415,73.909383,0217.4,bc12,7d,85,50,00,0e,1700,0e96,01dd,01af,0394,039c,0d2c,0ceb,0c4d,10bb,056d,04b0,0caf,0cb2,07fa,016c,04ad,0286,00aa,015e,006c,0.0,fc36,f666,fd7f,1.0,0.0,ffa7,000000ff,07ffffff
54,04/05/2021 21:59:28,18.900920,73.913392,0217.4,b812,7d,85,50,00,0e,1800,0eb2,01e2,01be,0394,039c,0d2d,0ceb,0c4f,10bb,056b,04af,0caf,0cb2,07f8,016c,04a5,0295,00aa,0171,006c,0.0,fc37,f666,fd7f,1.0,0.0,003c,000000ff,07ffffff


Dealing with RPM Column


In [ ]:
rpm_list = []

for i in dataset['RPM']:
  if i != 0:
    old_list = []
    for j in i:
      old_list.append(j)
    rearrange = ' '+ old_list[2] + old_list[3] + old_list[0] + old_list[1]
    rpm_list.append((int(rearrange,16)*0.125))
  elif i == 0:
    rpm_list.append(0)

In [ ]:
dataset.drop('RPM',axis=1)
dataset['RPM'] = (rpm_list)

In [ ]:
dataset.iloc[50:55]

,Time,Latitude,Longitude,Altitude,RPM,Driver Demand Torque (%),Engine Load (%),Engine Torque Mode,TPS (%),Percent Load Curret Speed,Fuel Rate (L-Hr),Vehicle Speed,Inj Q Cur (mg-st),Inj Q Tor (mg-st),Boost Pressure (mBar),Atmospheric Pressure (mBar),Coolant Temperature (*C),Oil Temperature (*C),Boost Temperature (*C),Oil Pressure (mBar),Battery Voltage (V),Cam Speed (rpm),Rail Pressure (mBar),Rail Pressure set (mBar),MU PWM (%),MU Vol (mm3-s),Torque Rat,Torque (Nm),TQ Limit Set,Main Injection (mg-st),Pilot Injection (mg-st),Pos 2 Injector (mg-st),EGR Prop (%),EGR Pos D (%),EGR Pos A (%),Clutch Switch,Brake Switch,Engine Grad (rpm),param1,param2
50,04/05/2021 21:59:24,18.900668,73.909515,0217.4,599.0,7d,85,40,00,0f,1800,0e94,01d7,0135,039b,039c,0d2a,0ce8,0c49,10cf,0575,04b2,0bd3,0cb2,07f3,0176,04bc,02a0,00aa,0171,006d,0.0,fc35,f666,fd7f,1.0,0.0,ffc5,000001ff,07ffffff
51,04/05/2021 21:59:25,18.900592,73.909465,0217.4,600.0,7d,85,00,00,0e,1800,0e99,01bc,0197,0395,039c,0d2c,0ce9,0c4b,10bb,056d,04b0,0cdb,0cb2,07f6,0162,04a1,0293,00aa,0170,006c,0.0,fc35,f666,fd7f,1.0,0.0,003c,000000ff,07ffffff
52,04/05/2021 21:59:25,18.900592,73.909465,0217.4,602.0,7d,85,00,00,0d,1800,0e99,01bc,0197,0395,039c,0d2d,0cea,0c4c,10ae,056c,04b0,0c99,0cb2,07f5,016e,0499,0283,00aa,016a,006c,0.0,fc35,f666,fd7f,1.0,0.0,001e,0000007f,07ffffff
53,04/05/2021 21:59:27,18.900415,73.909383,0217.4,599.5,7d,85,50,00,0e,1700,0e96,01dd,01af,0394,039c,0d2c,0ceb,0c4d,10bb,056d,04b0,0caf,0cb2,07fa,016c,04ad,0286,00aa,015e,006c,0.0,fc36,f666,fd7f,1.0,0.0,ffa7,000000ff,07ffffff
54,04/05/2021 21:59:28,18.900920,73.913392,0217.4,599.0,7d,85,50,00,0e,1800,0eb2,01e2,01be,0394,039c,0d2d,0ceb,0c4f,10bb,056b,04af,0caf,0cb2,07f8,016c,04a5,0295,00aa,0171,006c,0.0,fc37,f666,fd7f,1.0,0.0,003c,000000ff,07ffffff


Dealing with Driver Demand Torque 

In [ ]:
ddt = []
for i in dataset['Driver Demand Torque (%)']:
  if i != 0:
    ddt.append((int(i,16))+(-125))
  else:
    ddt.append(0)

In [ ]:
dataset.drop('Driver Demand Torque (%)',axis=1)
dataset['Driver Demand Torque (%)'] = (ddt)

In [ ]:
dataset.iloc[50:55]

,Time,Latitude,Longitude,Altitude,RPM,Driver Demand Torque (%),Engine Load (%),Engine Torque Mode,TPS (%),Percent Load Curret Speed,Fuel Rate (L-Hr),Vehicle Speed,Inj Q Cur (mg-st),Inj Q Tor (mg-st),Boost Pressure (mBar),Atmospheric Pressure (mBar),Coolant Temperature (*C),Oil Temperature (*C),Boost Temperature (*C),Oil Pressure (mBar),Battery Voltage (V),Cam Speed (rpm),Rail Pressure (mBar),Rail Pressure set (mBar),MU PWM (%),MU Vol (mm3-s),Torque Rat,Torque (Nm),TQ Limit Set,Main Injection (mg-st),Pilot Injection (mg-st),Pos 2 Injector (mg-st),EGR Prop (%),EGR Pos D (%),EGR Pos A (%),Clutch Switch,Brake Switch,Engine Grad (rpm),param1,param2
50,04/05/2021 21:59:24,18.900668,73.909515,0217.4,599.0,0,85,40,00,0f,1800,0e94,01d7,0135,039b,039c,0d2a,0ce8,0c49,10cf,0575,04b2,0bd3,0cb2,07f3,0176,04bc,02a0,00aa,0171,006d,0.0,fc35,f666,fd7f,1.0,0.0,ffc5,000001ff,07ffffff
51,04/05/2021 21:59:25,18.900592,73.909465,0217.4,600.0,0,85,00,00,0e,1800,0e99,01bc,0197,0395,039c,0d2c,0ce9,0c4b,10bb,056d,04b0,0cdb,0cb2,07f6,0162,04a1,0293,00aa,0170,006c,0.0,fc35,f666,fd7f,1.0,0.0,003c,000000ff,07ffffff
52,04/05/2021 21:59:25,18.900592,73.909465,0217.4,602.0,0,85,00,00,0d,1800,0e99,01bc,0197,0395,039c,0d2d,0cea,0c4c,10ae,056c,04b0,0c99,0cb2,07f5,016e,0499,0283,00aa,016a,006c,0.0,fc35,f666,fd7f,1.0,0.0,001e,0000007f,07ffffff
53,04/05/2021 21:59:27,18.900415,73.909383,0217.4,599.5,0,85,50,00,0e,1700,0e96,01dd,01af,0394,039c,0d2c,0ceb,0c4d,10bb,056d,04b0,0caf,0cb2,07fa,016c,04ad,0286,00aa,015e,006c,0.0,fc36,f666,fd7f,1.0,0.0,ffa7,000000ff,07ffffff
54,04/05/2021 21:59:28,18.900920,73.913392,0217.4,599.0,0,85,50,00,0e,1800,0eb2,01e2,01be,0394,039c,0d2d,0ceb,0c4f,10bb,056b,04af,0caf,0cb2,07f8,016c,04a5,0295,00aa,0171,006c,0.0,fc37,f666,fd7f,1.0,0.0,003c,000000ff,07ffffff


Dealing with Engine Load

In [ ]:
engine_load = []

for i in dataset['Engine Load (%)']:
    if i != 0:
      engine_load.append((int(i,16))+(-125))
    else:
      engine_load.append(0)

In [ ]:
dataset.drop('Engine Load (%)',axis=1)
dataset['Engine Load (%)'] = (engine_load)

In [ ]:
dataset.iloc[50:55]

,Time,Latitude,Longitude,Altitude,RPM,Driver Demand Torque (%),Engine Load (%),Engine Torque Mode,TPS (%),Percent Load Curret Speed,Fuel Rate (L-Hr),Vehicle Speed,Inj Q Cur (mg-st),Inj Q Tor (mg-st),Boost Pressure (mBar),Atmospheric Pressure (mBar),Coolant Temperature (*C),Oil Temperature (*C),Boost Temperature (*C),Oil Pressure (mBar),Battery Voltage (V),Cam Speed (rpm),Rail Pressure (mBar),Rail Pressure set (mBar),MU PWM (%),MU Vol (mm3-s),Torque Rat,Torque (Nm),TQ Limit Set,Main Injection (mg-st),Pilot Injection (mg-st),Pos 2 Injector (mg-st),EGR Prop (%),EGR Pos D (%),EGR Pos A (%),Clutch Switch,Brake Switch,Engine Grad (rpm),param1,param2
50,04/05/2021 21:59:24,18.900668,73.909515,0217.4,599.0,0,8,40,00,0f,1800,0e94,01d7,0135,039b,039c,0d2a,0ce8,0c49,10cf,0575,04b2,0bd3,0cb2,07f3,0176,04bc,02a0,00aa,0171,006d,0.0,fc35,f666,fd7f,1.0,0.0,ffc5,000001ff,07ffffff
51,04/05/2021 21:59:25,18.900592,73.909465,0217.4,600.0,0,8,00,00,0e,1800,0e99,01bc,0197,0395,039c,0d2c,0ce9,0c4b,10bb,056d,04b0,0cdb,0cb2,07f6,0162,04a1,0293,00aa,0170,006c,0.0,fc35,f666,fd7f,1.0,0.0,003c,000000ff,07ffffff
52,04/05/2021 21:59:25,18.900592,73.909465,0217.4,602.0,0,8,00,00,0d,1800,0e99,01bc,0197,0395,039c,0d2d,0cea,0c4c,10ae,056c,04b0,0c99,0cb2,07f5,016e,0499,0283,00aa,016a,006c,0.0,fc35,f666,fd7f,1.0,0.0,001e,0000007f,07ffffff
53,04/05/2021 21:59:27,18.900415,73.909383,0217.4,599.5,0,8,50,00,0e,1700,0e96,01dd,01af,0394,039c,0d2c,0ceb,0c4d,10bb,056d,04b0,0caf,0cb2,07fa,016c,04ad,0286,00aa,015e,006c,0.0,fc36,f666,fd7f,1.0,0.0,ffa7,000000ff,07ffffff
54,04/05/2021 21:59:28,18.900920,73.913392,0217.4,599.0,0,8,50,00,0e,1800,0eb2,01e2,01be,0394,039c,0d2d,0ceb,0c4f,10bb,056b,04af,0caf,0cb2,07f8,016c,04a5,0295,00aa,0171,006c,0.0,fc37,f666,fd7f,1.0,0.0,003c,000000ff,07ffffff


Dealing with Engine Torque Mode

In [ ]:
ETMode = []

for i in dataset['Engine Torque Mode']:
  if i != 0:
    ETMode.append(int(i,16)-125)
  else:
    ETMode.append(0)  

In [ ]:
dataset.drop('Engine Torque Mode',axis=1)
dataset['Engine Torque Mode'] = (ETMode)

In [ ]:
dataset.iloc[50:55]

,Time,Latitude,Longitude,Altitude,RPM,Driver Demand Torque (%),Engine Load (%),Engine Torque Mode,TPS (%),Percent Load Curret Speed,Fuel Rate (L-Hr),Vehicle Speed,Inj Q Cur (mg-st),Inj Q Tor (mg-st),Boost Pressure (mBar),Atmospheric Pressure (mBar),Coolant Temperature (*C),Oil Temperature (*C),Boost Temperature (*C),Oil Pressure (mBar),Battery Voltage (V),Cam Speed (rpm),Rail Pressure (mBar),Rail Pressure set (mBar),MU PWM (%),MU Vol (mm3-s),Torque Rat,Torque (Nm),TQ Limit Set,Main Injection (mg-st),Pilot Injection (mg-st),Pos 2 Injector (mg-st),EGR Prop (%),EGR Pos D (%),EGR Pos A (%),Clutch Switch,Brake Switch,Engine Grad (rpm),param1,param2
50,04/05/2021 21:59:24,18.900668,73.909515,0217.4,599.0,0,8,-61,00,0f,1800,0e94,01d7,0135,039b,039c,0d2a,0ce8,0c49,10cf,0575,04b2,0bd3,0cb2,07f3,0176,04bc,02a0,00aa,0171,006d,0.0,fc35,f666,fd7f,1.0,0.0,ffc5,000001ff,07ffffff
51,04/05/2021 21:59:25,18.900592,73.909465,0217.4,600.0,0,8,-125,00,0e,1800,0e99,01bc,0197,0395,039c,0d2c,0ce9,0c4b,10bb,056d,04b0,0cdb,0cb2,07f6,0162,04a1,0293,00aa,0170,006c,0.0,fc35,f666,fd7f,1.0,0.0,003c,000000ff,07ffffff
52,04/05/2021 21:59:25,18.900592,73.909465,0217.4,602.0,0,8,-125,00,0d,1800,0e99,01bc,0197,0395,039c,0d2d,0cea,0c4c,10ae,056c,04b0,0c99,0cb2,07f5,016e,0499,0283,00aa,016a,006c,0.0,fc35,f666,fd7f,1.0,0.0,001e,0000007f,07ffffff
53,04/05/2021 21:59:27,18.900415,73.909383,0217.4,599.5,0,8,-45,00,0e,1700,0e96,01dd,01af,0394,039c,0d2c,0ceb,0c4d,10bb,056d,04b0,0caf,0cb2,07fa,016c,04ad,0286,00aa,015e,006c,0.0,fc36,f666,fd7f,1.0,0.0,ffa7,000000ff,07ffffff
54,04/05/2021 21:59:28,18.900920,73.913392,0217.4,599.0,0,8,-45,00,0e,1800,0eb2,01e2,01be,0394,039c,0d2d,0ceb,0c4f,10bb,056b,04af,0caf,0cb2,07f8,016c,04a5,0295,00aa,0171,006c,0.0,fc37,f666,fd7f,1.0,0.0,003c,000000ff,07ffffff


Dealing with TPS(%)

In [ ]:
TPS = []

for i in dataset['TPS (%)']:
  if i != 0:
    TPS.append((int(i,16))*0.4)
  else:
    TPS.append(0)

In [ ]:
dataset.drop('TPS (%)',axis=1)
dataset['TPS (%)'] = (TPS)

In [ ]:
dataset.iloc[1000:1005]

,Time,Latitude,Longitude,Altitude,RPM,Driver Demand Torque (%),Engine Load (%),Engine Torque Mode,TPS (%),Percent Load Curret Speed,Fuel Rate (L-Hr),Vehicle Speed,Inj Q Cur (mg-st),Inj Q Tor (mg-st),Boost Pressure (mBar),Atmospheric Pressure (mBar),Coolant Temperature (*C),Oil Temperature (*C),Boost Temperature (*C),Oil Pressure (mBar),Battery Voltage (V),Cam Speed (rpm),Rail Pressure (mBar),Rail Pressure set (mBar),MU PWM (%),MU Vol (mm3-s),Torque Rat,Torque (Nm),TQ Limit Set,Main Injection (mg-st),Pilot Injection (mg-st),Pos 2 Injector (mg-st),EGR Prop (%),EGR Pos D (%),EGR Pos A (%),Clutch Switch,Brake Switch,Engine Grad (rpm),param1,param2
1000,04/05/2021 22:15:13,18.806947,73.873157,0606.6,599.5,0,7,-109,0.0,0c,1400,145e,0182,018c,03b2,03ac,0dfe,0dcb,0c16,0c8e,0567,04b0,0c84,0cb2,080a,0181,03f1,0219,00aa,0124,0064,0.0,fc34,f666,fd7f,1.0,0.0,0000,000001ff,07ffffff
1001,04/05/2021 22:15:14,18.806827,73.873113,0606.6,600.0,0,7,-77,0.0,0c,1300,1427,0195,018a,03b2,03ac,0dfe,0dcb,0c16,0c87,0565,04af,0cc5,0cb2,0816,0171,03e3,0219,00aa,0124,0064,0.0,fc33,f666,fd7f,1.0,0.0,0000,000000ff,07ffffff
1002,04/05/2021 22:15:15,18.806705,73.873073,0606.6,598.5,0,7,-77,0.0,0c,1400,1427,0195,018a,03b2,03ac,0dfe,0dcb,0c16,0c87,0565,04af,0c99,0cb2,0814,017d,03d2,0219,00aa,0124,0064,0.0,fc34,f666,fd7f,1.0,0.0,ffe3,000000ff,07ffffff
1003,04/05/2021 22:15:16,18.806585,73.873037,0606.6,599.0,0,7,-77,0.0,0c,1500,1426,0197,018c,03b2,03ac,0dfe,0dca,0c16,0c87,0569,04ae,0cdb,0cb2,0812,0175,0422,0232,00aa,0136,0064,0.0,fc33,f666,fd7f,1.0,0.0,001e,000000ff,07ffffff
1004,04/05/2021 22:15:17,18.806465,73.873000,0606.6,599.0,0,7,-77,0.0,0c,1400,13f3,0197,018c,03b1,03ac,0dfe,0dcb,0c16,0c8e,0563,04af,0c99,0cb2,0814,017e,0411,0220,00aa,0129,0064,0.0,fc33,f666,fd7f,1.0,0.0,ffe3,000000ff,07ffffff


Percenrage load current speed

In [ ]:
LCS = []

for i in dataset['Percent Load Curret Speed']:
  if i != 0:
    LCS.append((int(i,16))*1)
  else:
    LCS.append(0)

In [ ]:
dataset.drop('Percent Load Curret Speed',axis=1)
dataset['Percent Load Curret Speed'] = (LCS)

In [ ]:
dataset.iloc[50:55]

,Time,Latitude,Longitude,Altitude,RPM,Driver Demand Torque (%),Engine Load (%),Engine Torque Mode,TPS (%),Percent Load Curret Speed,Fuel Rate (L-Hr),Vehicle Speed,Inj Q Cur (mg-st),Inj Q Tor (mg-st),Boost Pressure (mBar),Atmospheric Pressure (mBar),Coolant Temperature (*C),Oil Temperature (*C),Boost Temperature (*C),Oil Pressure (mBar),Battery Voltage (V),Cam Speed (rpm),Rail Pressure (mBar),Rail Pressure set (mBar),MU PWM (%),MU Vol (mm3-s),Torque Rat,Torque (Nm),TQ Limit Set,Main Injection (mg-st),Pilot Injection (mg-st),Pos 2 Injector (mg-st),EGR Prop (%),EGR Pos D (%),EGR Pos A (%),Clutch Switch,Brake Switch,Engine Grad (rpm),param1,param2
50,04/05/2021 21:59:24,18.900668,73.909515,0217.4,599.0,0,8,-61,0.0,15,1800,0e94,01d7,0135,039b,039c,0d2a,0ce8,0c49,10cf,0575,04b2,0bd3,0cb2,07f3,0176,04bc,02a0,00aa,0171,006d,0.0,fc35,f666,fd7f,1.0,0.0,ffc5,000001ff,07ffffff
51,04/05/2021 21:59:25,18.900592,73.909465,0217.4,600.0,0,8,-125,0.0,14,1800,0e99,01bc,0197,0395,039c,0d2c,0ce9,0c4b,10bb,056d,04b0,0cdb,0cb2,07f6,0162,04a1,0293,00aa,0170,006c,0.0,fc35,f666,fd7f,1.0,0.0,003c,000000ff,07ffffff
52,04/05/2021 21:59:25,18.900592,73.909465,0217.4,602.0,0,8,-125,0.0,13,1800,0e99,01bc,0197,0395,039c,0d2d,0cea,0c4c,10ae,056c,04b0,0c99,0cb2,07f5,016e,0499,0283,00aa,016a,006c,0.0,fc35,f666,fd7f,1.0,0.0,001e,0000007f,07ffffff
53,04/05/2021 21:59:27,18.900415,73.909383,0217.4,599.5,0,8,-45,0.0,14,1700,0e96,01dd,01af,0394,039c,0d2c,0ceb,0c4d,10bb,056d,04b0,0caf,0cb2,07fa,016c,04ad,0286,00aa,015e,006c,0.0,fc36,f666,fd7f,1.0,0.0,ffa7,000000ff,07ffffff
54,04/05/2021 21:59:28,18.900920,73.913392,0217.4,599.0,0,8,-45,0.0,14,1800,0eb2,01e2,01be,0394,039c,0d2d,0ceb,0c4f,10bb,056b,04af,0caf,0cb2,07f8,016c,04a5,0295,00aa,0171,006c,0.0,fc37,f666,fd7f,1.0,0.0,003c,000000ff,07ffffff


Dealing with Fuel Rate (L-Hr)



In [ ]:
FRate = []

for i in dataset['Fuel Rate (L-Hr)']:
  if i != 0:
    FRate.append((int(i,16))*0.05)
  else:
    FRate.append(0)

In [ ]:
dd = []
for i in FRate:
  dd.append(float(i))

In [ ]:
FRate_featurescaled = []
FRateOldRange = (max(dd) - min(dd))  
FRateNewRange = (40 - 0)  
for i in dd:
  FRateNewValue = (((i - 0) * FRateNewRange) / FRateOldRange) + 0
  FRate_featurescaled.append(FRateNewValue)

In [ ]:
dataset.drop('Fuel Rate (L-Hr)',axis=1)
dataset['Fuel Rate (L-Hr)'] = (FRate_featurescaled)

In [ ]:
dataset.iloc[50:55]

,Time,Latitude,Longitude,Altitude,RPM,Driver Demand Torque (%),Engine Load (%),Engine Torque Mode,TPS (%),Percent Load Curret Speed,Fuel Rate (L-Hr),Vehicle Speed,Inj Q Cur (mg-st),Inj Q Tor (mg-st),Boost Pressure (mBar),Atmospheric Pressure (mBar),Coolant Temperature (*C),Oil Temperature (*C),Boost Temperature (*C),Oil Pressure (mBar),Battery Voltage (V),Cam Speed (rpm),Rail Pressure (mBar),Rail Pressure set (mBar),MU PWM (%),MU Vol (mm3-s),Torque Rat,Torque (Nm),TQ Limit Set,Main Injection (mg-st),Pilot Injection (mg-st),Pos 2 Injector (mg-st),EGR Prop (%),EGR Pos D (%),EGR Pos A (%),Clutch Switch,Brake Switch,Engine Grad (rpm),param1,param2
50,04/05/2021 21:59:24,18.900668,73.909515,0217.4,599.0,0,8,-61,0.0,15,3.764706,0e94,01d7,0135,039b,039c,0d2a,0ce8,0c49,10cf,0575,04b2,0bd3,0cb2,07f3,0176,04bc,02a0,00aa,0171,006d,0.0,fc35,f666,fd7f,1.0,0.0,ffc5,000001ff,07ffffff
51,04/05/2021 21:59:25,18.900592,73.909465,0217.4,600.0,0,8,-125,0.0,14,3.764706,0e99,01bc,0197,0395,039c,0d2c,0ce9,0c4b,10bb,056d,04b0,0cdb,0cb2,07f6,0162,04a1,0293,00aa,0170,006c,0.0,fc35,f666,fd7f,1.0,0.0,003c,000000ff,07ffffff
52,04/05/2021 21:59:25,18.900592,73.909465,0217.4,602.0,0,8,-125,0.0,13,3.764706,0e99,01bc,0197,0395,039c,0d2d,0cea,0c4c,10ae,056c,04b0,0c99,0cb2,07f5,016e,0499,0283,00aa,016a,006c,0.0,fc35,f666,fd7f,1.0,0.0,001e,0000007f,07ffffff
53,04/05/2021 21:59:27,18.900415,73.909383,0217.4,599.5,0,8,-45,0.0,14,3.607843,0e96,01dd,01af,0394,039c,0d2c,0ceb,0c4d,10bb,056d,04b0,0caf,0cb2,07fa,016c,04ad,0286,00aa,015e,006c,0.0,fc36,f666,fd7f,1.0,0.0,ffa7,000000ff,07ffffff
54,04/05/2021 21:59:28,18.900920,73.913392,0217.4,599.0,0,8,-45,0.0,14,3.764706,0eb2,01e2,01be,0394,039c,0d2d,0ceb,0c4f,10bb,056b,04af,0caf,0cb2,07f8,016c,04a5,0295,00aa,0171,006c,0.0,fc37,f666,fd7f,1.0,0.0,003c,000000ff,07ffffff


Dealing with Vehicle Speed

In [ ]:
VSpeed = []

for i in dataset['Vehicle Speed ']:
  if i != 0:
    VSpeed.append((int(i,16))*0.01)
  else:
    VSpeed.append(0)

In [ ]:
dataset.drop('Vehicle Speed ',axis=1)
dataset['Vehicle Speed '] = (VSpeed)

In [ ]:
dataset.iloc[50:55]

,Time,Latitude,Longitude,Altitude,RPM,Driver Demand Torque (%),Engine Load (%),Engine Torque Mode,TPS (%),Percent Load Curret Speed,Fuel Rate (L-Hr),Vehicle Speed,Inj Q Cur (mg-st),Inj Q Tor (mg-st),Boost Pressure (mBar),Atmospheric Pressure (mBar),Coolant Temperature (*C),Oil Temperature (*C),Boost Temperature (*C),Oil Pressure (mBar),Battery Voltage (V),Cam Speed (rpm),Rail Pressure (mBar),Rail Pressure set (mBar),MU PWM (%),MU Vol (mm3-s),Torque Rat,Torque (Nm),TQ Limit Set,Main Injection (mg-st),Pilot Injection (mg-st),Pos 2 Injector (mg-st),EGR Prop (%),EGR Pos D (%),EGR Pos A (%),Clutch Switch,Brake Switch,Engine Grad (rpm),param1,param2
50,04/05/2021 21:59:24,18.900668,73.909515,0217.4,599.0,0,8,-61,0.0,15,3.764706,37.32,01d7,0135,039b,039c,0d2a,0ce8,0c49,10cf,0575,04b2,0bd3,0cb2,07f3,0176,04bc,02a0,00aa,0171,006d,0.0,fc35,f666,fd7f,1.0,0.0,ffc5,000001ff,07ffffff
51,04/05/2021 21:59:25,18.900592,73.909465,0217.4,600.0,0,8,-125,0.0,14,3.764706,37.37,01bc,0197,0395,039c,0d2c,0ce9,0c4b,10bb,056d,04b0,0cdb,0cb2,07f6,0162,04a1,0293,00aa,0170,006c,0.0,fc35,f666,fd7f,1.0,0.0,003c,000000ff,07ffffff
52,04/05/2021 21:59:25,18.900592,73.909465,0217.4,602.0,0,8,-125,0.0,13,3.764706,37.37,01bc,0197,0395,039c,0d2d,0cea,0c4c,10ae,056c,04b0,0c99,0cb2,07f5,016e,0499,0283,00aa,016a,006c,0.0,fc35,f666,fd7f,1.0,0.0,001e,0000007f,07ffffff
53,04/05/2021 21:59:27,18.900415,73.909383,0217.4,599.5,0,8,-45,0.0,14,3.607843,37.34,01dd,01af,0394,039c,0d2c,0ceb,0c4d,10bb,056d,04b0,0caf,0cb2,07fa,016c,04ad,0286,00aa,015e,006c,0.0,fc36,f666,fd7f,1.0,0.0,ffa7,000000ff,07ffffff
54,04/05/2021 21:59:28,18.900920,73.913392,0217.4,599.0,0,8,-45,0.0,14,3.764706,37.62,01e2,01be,0394,039c,0d2d,0ceb,0c4f,10bb,056b,04af,0caf,0cb2,07f8,016c,04a5,0295,00aa,0171,006c,0.0,fc37,f666,fd7f,1.0,0.0,003c,000000ff,07ffffff


Dealing with Inj Q Cur (mg-st)

In [ ]:
InjCur = []

for i in dataset['Inj Q Cur (mg-st)']:
  if i != 0:
    InjCur.append((int(i,16))*0.02)
  else:
    InjCur.append(0)

In [ ]:
dataset.drop('Inj Q Cur (mg-st)',axis=1)
dataset['Inj Q Cur (mg-st)'] = (InjCur)

In [ ]:
dataset.iloc[50:55]

,Time,Latitude,Longitude,Altitude,RPM,Driver Demand Torque (%),Engine Load (%),Engine Torque Mode,TPS (%),Percent Load Curret Speed,Fuel Rate (L-Hr),Vehicle Speed,Inj Q Cur (mg-st),Inj Q Tor (mg-st),Boost Pressure (mBar),Atmospheric Pressure (mBar),Coolant Temperature (*C),Oil Temperature (*C),Boost Temperature (*C),Oil Pressure (mBar),Battery Voltage (V),Cam Speed (rpm),Rail Pressure (mBar),Rail Pressure set (mBar),MU PWM (%),MU Vol (mm3-s),Torque Rat,Torque (Nm),TQ Limit Set,Main Injection (mg-st),Pilot Injection (mg-st),Pos 2 Injector (mg-st),EGR Prop (%),EGR Pos D (%),EGR Pos A (%),Clutch Switch,Brake Switch,Engine Grad (rpm),param1,param2
50,04/05/2021 21:59:24,18.900668,73.909515,0217.4,599.0,0,8,-61,0.0,15,3.764706,37.32,9.42,0135,039b,039c,0d2a,0ce8,0c49,10cf,0575,04b2,0bd3,0cb2,07f3,0176,04bc,02a0,00aa,0171,006d,0.0,fc35,f666,fd7f,1.0,0.0,ffc5,000001ff,07ffffff
51,04/05/2021 21:59:25,18.900592,73.909465,0217.4,600.0,0,8,-125,0.0,14,3.764706,37.37,8.88,0197,0395,039c,0d2c,0ce9,0c4b,10bb,056d,04b0,0cdb,0cb2,07f6,0162,04a1,0293,00aa,0170,006c,0.0,fc35,f666,fd7f,1.0,0.0,003c,000000ff,07ffffff
52,04/05/2021 21:59:25,18.900592,73.909465,0217.4,602.0,0,8,-125,0.0,13,3.764706,37.37,8.88,0197,0395,039c,0d2d,0cea,0c4c,10ae,056c,04b0,0c99,0cb2,07f5,016e,0499,0283,00aa,016a,006c,0.0,fc35,f666,fd7f,1.0,0.0,001e,0000007f,07ffffff
53,04/05/2021 21:59:27,18.900415,73.909383,0217.4,599.5,0,8,-45,0.0,14,3.607843,37.34,9.54,01af,0394,039c,0d2c,0ceb,0c4d,10bb,056d,04b0,0caf,0cb2,07fa,016c,04ad,0286,00aa,015e,006c,0.0,fc36,f666,fd7f,1.0,0.0,ffa7,000000ff,07ffffff
54,04/05/2021 21:59:28,18.900920,73.913392,0217.4,599.0,0,8,-45,0.0,14,3.764706,37.62,9.64,01be,0394,039c,0d2d,0ceb,0c4f,10bb,056b,04af,0caf,0cb2,07f8,016c,04a5,0295,00aa,0171,006c,0.0,fc37,f666,fd7f,1.0,0.0,003c,000000ff,07ffffff


Dealing with Inj Q Tor (mg-st)

In [ ]:
InjTor = []

for i in dataset['Inj Q Tor (mg-st)']:
  if i != 0:
    InjTor.append((int(i,16))*0.02)
  else:
    InjTor.append(0)      

In [ ]:
dataset.drop('Inj Q Tor (mg-st)',axis=1)
dataset['Inj Q Tor (mg-st)'] = (InjTor)

In [ ]:
dataset.iloc[50:55]

,Time,Latitude,Longitude,Altitude,RPM,Driver Demand Torque (%),Engine Load (%),Engine Torque Mode,TPS (%),Percent Load Curret Speed,Fuel Rate (L-Hr),Vehicle Speed,Inj Q Cur (mg-st),Inj Q Tor (mg-st),Boost Pressure (mBar),Atmospheric Pressure (mBar),Coolant Temperature (*C),Oil Temperature (*C),Boost Temperature (*C),Oil Pressure (mBar),Battery Voltage (V),Cam Speed (rpm),Rail Pressure (mBar),Rail Pressure set (mBar),MU PWM (%),MU Vol (mm3-s),Torque Rat,Torque (Nm),TQ Limit Set,Main Injection (mg-st),Pilot Injection (mg-st),Pos 2 Injector (mg-st),EGR Prop (%),EGR Pos D (%),EGR Pos A (%),Clutch Switch,Brake Switch,Engine Grad (rpm),param1,param2
0,04/05/2021 29:29:45,0.0,0.0,0,554.0,0,8,-29,0.0,14,3.450980,0.0,9.58,9.04,0392,039c,0d30,0cd3,0cc6,1135,0552,044f,0c84,0cb2,081a,014a,04a9,0293,00aa,017b,006c,0.0,fc1f,f666,fd64,1.0,0.0,001b,000000ff,07ffffff
1,04/05/2021 29:29:46,0.0,0.0,0,550.0,0,8,-29,0.0,15,3.607843,0.0,9.90,9.32,0391,039c,0d30,0cd3,0cbe,1127,0558,044c,0c99,0cb2,081c,013f,04ce,0296,00aa,0177,006c,0.0,fc28,f666,fd64,1.0,0.0,0000,000000ff,07ffffff
2,04/05/2021 29:29:47,0.0,0.0,0,550.5,0,8,-45,0.0,14,3.450980,0.0,9.62,9.48,0390,039c,0d2f,0cd3,0cb8,10f8,0569,044c,0caf,0cb2,0802,0149,04ea,029c,00aa,017c,006c,0.0,fc2f,f666,fd64,1.0,0.0,0000,000000ff,07ffffff
3,04/05/2021 29:29:48,0.0,0.0,0,550.0,0,8,-29,0.0,15,3.607843,0.0,9.70,9.56,0390,039c,0d2e,0cd4,0cb3,1113,0570,044c,0caf,0cb2,07ef,014d,04c1,0298,00aa,0179,006c,0.0,fc38,f666,fd64,1.0,0.0,0000,000001ff,07ffffff
4,04/05/2021 29:29:49,0.0,0.0,0,550.0,0,8,-45,0.0,14,3.450980,0.0,9.56,9.58,038f,039b,0d2d,0cd4,0cae,10cf,0574,044d,0caf,0cb2,07f1,014e,04b2,0285,00aa,0164,006c,0.0,fc38,f666,fd64,1.0,0.0,ffe5,000000ff,07ffffff


Dealing with Boost Pressure

In [ ]:
BPressure = []

for i in dataset['Boost Pressure (mBar)']:
  if i != 0:
    BPressure.append((int(i,16))*1.00)
  else:
    BPressure.append(0)

In [ ]:
dataset.drop('Boost Pressure (mBar)',axis=1)
dataset['Boost Pressure (mBar)'] = (BPressure)

In [ ]:
dataset.head()

,Time,Latitude,Longitude,Altitude,RPM,Driver Demand Torque (%),Engine Load (%),Engine Torque Mode,TPS (%),Percent Load Curret Speed,Fuel Rate (L-Hr),Vehicle Speed,Inj Q Cur (mg-st),Inj Q Tor (mg-st),Boost Pressure (mBar),Atmospheric Pressure (mBar),Coolant Temperature (*C),Oil Temperature (*C),Boost Temperature (*C),Oil Pressure (mBar),Battery Voltage (V),Cam Speed (rpm),Rail Pressure (mBar),Rail Pressure set (mBar),MU PWM (%),MU Vol (mm3-s),Torque Rat,Torque (Nm),TQ Limit Set,Main Injection (mg-st),Pilot Injection (mg-st),Pos 2 Injector (mg-st),EGR Prop (%),EGR Pos D (%),EGR Pos A (%),Clutch Switch,Brake Switch,Engine Grad (rpm),param1,param2
0,04/05/2021 29:29:45,0.0,0.0,0,554.0,0,8,-29,0.0,14,3.450980,0.0,9.58,9.04,914.0,039c,0d30,0cd3,0cc6,1135,0552,044f,0c84,0cb2,081a,014a,04a9,0293,00aa,017b,006c,0.0,fc1f,f666,fd64,1.0,0.0,001b,000000ff,07ffffff
1,04/05/2021 29:29:46,0.0,0.0,0,550.0,0,8,-29,0.0,15,3.607843,0.0,9.90,9.32,913.0,039c,0d30,0cd3,0cbe,1127,0558,044c,0c99,0cb2,081c,013f,04ce,0296,00aa,0177,006c,0.0,fc28,f666,fd64,1.0,0.0,0000,000000ff,07ffffff
2,04/05/2021 29:29:47,0.0,0.0,0,550.5,0,8,-45,0.0,14,3.450980,0.0,9.62,9.48,912.0,039c,0d2f,0cd3,0cb8,10f8,0569,044c,0caf,0cb2,0802,0149,04ea,029c,00aa,017c,006c,0.0,fc2f,f666,fd64,1.0,0.0,0000,000000ff,07ffffff
3,04/05/2021 29:29:48,0.0,0.0,0,550.0,0,8,-29,0.0,15,3.607843,0.0,9.70,9.56,912.0,039c,0d2e,0cd4,0cb3,1113,0570,044c,0caf,0cb2,07ef,014d,04c1,0298,00aa,0179,006c,0.0,fc38,f666,fd64,1.0,0.0,0000,000001ff,07ffffff
4,04/05/2021 29:29:49,0.0,0.0,0,550.0,0,8,-45,0.0,14,3.450980,0.0,9.56,9.58,911.0,039b,0d2d,0cd4,0cae,10cf,0574,044d,0caf,0cb2,07f1,014e,04b2,0285,00aa,0164,006c,0.0,fc38,f666,fd64,1.0,0.0,ffe5,000000ff,07ffffff


Dealing with Atmospheric Pressure

In [ ]:
AtmPressure = []

for i in dataset['Atmospheric Pressure (mBar)']:
  if i != 0:
    AtmPressure.append((int(i,16))*1.00)
  else:
    AtmPressure.append(0)

In [ ]:
dataset.drop('Atmospheric Pressure (mBar)',axis=1)
dataset['Atmospheric Pressure (mBar)'] = (AtmPressure)

In [ ]:
dataset.iloc[50:55]

,Time,Latitude,Longitude,Altitude,RPM,Driver Demand Torque (%),Engine Load (%),Engine Torque Mode,TPS (%),Percent Load Curret Speed,Fuel Rate (L-Hr),Vehicle Speed,Inj Q Cur (mg-st),Inj Q Tor (mg-st),Boost Pressure (mBar),Atmospheric Pressure (mBar),Coolant Temperature (*C),Oil Temperature (*C),Boost Temperature (*C),Oil Pressure (mBar),Battery Voltage (V),Cam Speed (rpm),Rail Pressure (mBar),Rail Pressure set (mBar),MU PWM (%),MU Vol (mm3-s),Torque Rat,Torque (Nm),TQ Limit Set,Main Injection (mg-st),Pilot Injection (mg-st),Pos 2 Injector (mg-st),EGR Prop (%),EGR Pos D (%),EGR Pos A (%),Clutch Switch,Brake Switch,Engine Grad (rpm),param1,param2
50,04/05/2021 21:59:24,18.900668,73.909515,0217.4,599.0,0,8,-61,0.0,15,3.764706,37.32,9.42,6.18,923.0,924.0,0d2a,0ce8,0c49,10cf,0575,04b2,0bd3,0cb2,07f3,0176,04bc,02a0,00aa,0171,006d,0.0,fc35,f666,fd7f,1.0,0.0,ffc5,000001ff,07ffffff
51,04/05/2021 21:59:25,18.900592,73.909465,0217.4,600.0,0,8,-125,0.0,14,3.764706,37.37,8.88,8.14,917.0,924.0,0d2c,0ce9,0c4b,10bb,056d,04b0,0cdb,0cb2,07f6,0162,04a1,0293,00aa,0170,006c,0.0,fc35,f666,fd7f,1.0,0.0,003c,000000ff,07ffffff
52,04/05/2021 21:59:25,18.900592,73.909465,0217.4,602.0,0,8,-125,0.0,13,3.764706,37.37,8.88,8.14,917.0,924.0,0d2d,0cea,0c4c,10ae,056c,04b0,0c99,0cb2,07f5,016e,0499,0283,00aa,016a,006c,0.0,fc35,f666,fd7f,1.0,0.0,001e,0000007f,07ffffff
53,04/05/2021 21:59:27,18.900415,73.909383,0217.4,599.5,0,8,-45,0.0,14,3.607843,37.34,9.54,8.62,916.0,924.0,0d2c,0ceb,0c4d,10bb,056d,04b0,0caf,0cb2,07fa,016c,04ad,0286,00aa,015e,006c,0.0,fc36,f666,fd7f,1.0,0.0,ffa7,000000ff,07ffffff
54,04/05/2021 21:59:28,18.900920,73.913392,0217.4,599.0,0,8,-45,0.0,14,3.764706,37.62,9.64,8.92,916.0,924.0,0d2d,0ceb,0c4f,10bb,056b,04af,0caf,0cb2,07f8,016c,04a5,0295,00aa,0171,006c,0.0,fc37,f666,fd7f,1.0,0.0,003c,000000ff,07ffffff


Dealing with Coolant Temperature

In [ ]:
CTemp = []

for i in dataset['Coolant Temperature (*C)']:
  if i != 0:
    CTemp.append(((int(i,16))*0.10)+(-273))
  else:
    CTemp.append(0)

In [ ]:
dataset.drop('Coolant Temperature (*C)',axis=1)
dataset['Coolant Temperature (*C)'] = (CTemp)

In [ ]:
dataset.iloc[50:55]

,Time,Latitude,Longitude,Altitude,RPM,Driver Demand Torque (%),Engine Load (%),Engine Torque Mode,TPS (%),Percent Load Curret Speed,Fuel Rate (L-Hr),Vehicle Speed,Inj Q Cur (mg-st),Inj Q Tor (mg-st),Boost Pressure (mBar),Atmospheric Pressure (mBar),Coolant Temperature (*C),Oil Temperature (*C),Boost Temperature (*C),Oil Pressure (mBar),Battery Voltage (V),Cam Speed (rpm),Rail Pressure (mBar),Rail Pressure set (mBar),MU PWM (%),MU Vol (mm3-s),Torque Rat,Torque (Nm),TQ Limit Set,Main Injection (mg-st),Pilot Injection (mg-st),Pos 2 Injector (mg-st),EGR Prop (%),EGR Pos D (%),EGR Pos A (%),Clutch Switch,Brake Switch,Engine Grad (rpm),param1,param2
50,04/05/2021 21:59:24,18.900668,73.909515,0217.4,599.0,0,8,-61,0.0,15,3.764706,37.32,9.42,6.18,923.0,924.0,64.0,0ce8,0c49,10cf,0575,04b2,0bd3,0cb2,07f3,0176,04bc,02a0,00aa,0171,006d,0.0,fc35,f666,fd7f,1.0,0.0,ffc5,000001ff,07ffffff
51,04/05/2021 21:59:25,18.900592,73.909465,0217.4,600.0,0,8,-125,0.0,14,3.764706,37.37,8.88,8.14,917.0,924.0,64.2,0ce9,0c4b,10bb,056d,04b0,0cdb,0cb2,07f6,0162,04a1,0293,00aa,0170,006c,0.0,fc35,f666,fd7f,1.0,0.0,003c,000000ff,07ffffff
52,04/05/2021 21:59:25,18.900592,73.909465,0217.4,602.0,0,8,-125,0.0,13,3.764706,37.37,8.88,8.14,917.0,924.0,64.3,0cea,0c4c,10ae,056c,04b0,0c99,0cb2,07f5,016e,0499,0283,00aa,016a,006c,0.0,fc35,f666,fd7f,1.0,0.0,001e,0000007f,07ffffff
53,04/05/2021 21:59:27,18.900415,73.909383,0217.4,599.5,0,8,-45,0.0,14,3.607843,37.34,9.54,8.62,916.0,924.0,64.2,0ceb,0c4d,10bb,056d,04b0,0caf,0cb2,07fa,016c,04ad,0286,00aa,015e,006c,0.0,fc36,f666,fd7f,1.0,0.0,ffa7,000000ff,07ffffff
54,04/05/2021 21:59:28,18.900920,73.913392,0217.4,599.0,0,8,-45,0.0,14,3.764706,37.62,9.64,8.92,916.0,924.0,64.3,0ceb,0c4f,10bb,056b,04af,0caf,0cb2,07f8,016c,04a5,0295,00aa,0171,006c,0.0,fc37,f666,fd7f,1.0,0.0,003c,000000ff,07ffffff


Dealing with Oil Temperature

In [ ]:
OTemp = []

for i in dataset['Oil Temperature (*C)']:
  if i != 0:
    OTemp.append(((int(i,16))*0.10)+(-273))
  else:
    OTemp.append(0)

In [ ]:
dataset.drop('Oil Temperature (*C)',axis=1)
dataset['Oil Temperature (*C)'] = (OTemp)

In [ ]:
dataset.iloc[50:55]

,Time,Latitude,Longitude,Altitude,RPM,Driver Demand Torque (%),Engine Load (%),Engine Torque Mode,TPS (%),Percent Load Curret Speed,Fuel Rate (L-Hr),Vehicle Speed,Inj Q Cur (mg-st),Inj Q Tor (mg-st),Boost Pressure (mBar),Atmospheric Pressure (mBar),Coolant Temperature (*C),Oil Temperature (*C),Boost Temperature (*C),Oil Pressure (mBar),Battery Voltage (V),Cam Speed (rpm),Rail Pressure (mBar),Rail Pressure set (mBar),MU PWM (%),MU Vol (mm3-s),Torque Rat,Torque (Nm),TQ Limit Set,Main Injection (mg-st),Pilot Injection (mg-st),Pos 2 Injector (mg-st),EGR Prop (%),EGR Pos D (%),EGR Pos A (%),Clutch Switch,Brake Switch,Engine Grad (rpm),param1,param2
50,04/05/2021 21:59:24,18.900668,73.909515,0217.4,599.0,0,8,-61,0.0,15,3.764706,37.32,9.42,6.18,923.0,924.0,64.0,57.4,0c49,10cf,0575,04b2,0bd3,0cb2,07f3,0176,04bc,02a0,00aa,0171,006d,0.0,fc35,f666,fd7f,1.0,0.0,ffc5,000001ff,07ffffff
51,04/05/2021 21:59:25,18.900592,73.909465,0217.4,600.0,0,8,-125,0.0,14,3.764706,37.37,8.88,8.14,917.0,924.0,64.2,57.5,0c4b,10bb,056d,04b0,0cdb,0cb2,07f6,0162,04a1,0293,00aa,0170,006c,0.0,fc35,f666,fd7f,1.0,0.0,003c,000000ff,07ffffff
52,04/05/2021 21:59:25,18.900592,73.909465,0217.4,602.0,0,8,-125,0.0,13,3.764706,37.37,8.88,8.14,917.0,924.0,64.3,57.6,0c4c,10ae,056c,04b0,0c99,0cb2,07f5,016e,0499,0283,00aa,016a,006c,0.0,fc35,f666,fd7f,1.0,0.0,001e,0000007f,07ffffff
53,04/05/2021 21:59:27,18.900415,73.909383,0217.4,599.5,0,8,-45,0.0,14,3.607843,37.34,9.54,8.62,916.0,924.0,64.2,57.7,0c4d,10bb,056d,04b0,0caf,0cb2,07fa,016c,04ad,0286,00aa,015e,006c,0.0,fc36,f666,fd7f,1.0,0.0,ffa7,000000ff,07ffffff
54,04/05/2021 21:59:28,18.900920,73.913392,0217.4,599.0,0,8,-45,0.0,14,3.764706,37.62,9.64,8.92,916.0,924.0,64.3,57.7,0c4f,10bb,056b,04af,0caf,0cb2,07f8,016c,04a5,0295,00aa,0171,006c,0.0,fc37,f666,fd7f,1.0,0.0,003c,000000ff,07ffffff


Dealing with Boost Temperature

In [ ]:
BoostTemp = []

for i in dataset['Boost Temperature (*C)']:
  if i != 0:
    BoostTemp.append(((int(i,16))*0.10)+(-273))
  else:
    BoostTemp.append(0)

In [ ]:
dataset.drop('Boost Temperature (*C)',axis=1)
dataset['Boost Temperature (*C)'] = (BoostTemp)

In [ ]:
dataset.iloc[50:55]

,Time,Latitude,Longitude,Altitude,RPM,Driver Demand Torque (%),Engine Load (%),Engine Torque Mode,TPS (%),Percent Load Curret Speed,Fuel Rate (L-Hr),Vehicle Speed,Inj Q Cur (mg-st),Inj Q Tor (mg-st),Boost Pressure (mBar),Atmospheric Pressure (mBar),Coolant Temperature (*C),Oil Temperature (*C),Boost Temperature (*C),Oil Pressure (mBar),Battery Voltage (V),Cam Speed (rpm),Rail Pressure (mBar),Rail Pressure set (mBar),MU PWM (%),MU Vol (mm3-s),Torque Rat,Torque (Nm),TQ Limit Set,Main Injection (mg-st),Pilot Injection (mg-st),Pos 2 Injector (mg-st),EGR Prop (%),EGR Pos D (%),EGR Pos A (%),Clutch Switch,Brake Switch,Engine Grad (rpm),param1,param2
50,04/05/2021 21:59:24,18.900668,73.909515,0217.4,599.0,0,8,-61,0.0,15,3.764706,37.32,9.42,6.18,923.0,924.0,64.0,57.4,41.5,10cf,0575,04b2,0bd3,0cb2,07f3,0176,04bc,02a0,00aa,0171,006d,0.0,fc35,f666,fd7f,1.0,0.0,ffc5,000001ff,07ffffff
51,04/05/2021 21:59:25,18.900592,73.909465,0217.4,600.0,0,8,-125,0.0,14,3.764706,37.37,8.88,8.14,917.0,924.0,64.2,57.5,41.7,10bb,056d,04b0,0cdb,0cb2,07f6,0162,04a1,0293,00aa,0170,006c,0.0,fc35,f666,fd7f,1.0,0.0,003c,000000ff,07ffffff
52,04/05/2021 21:59:25,18.900592,73.909465,0217.4,602.0,0,8,-125,0.0,13,3.764706,37.37,8.88,8.14,917.0,924.0,64.3,57.6,41.8,10ae,056c,04b0,0c99,0cb2,07f5,016e,0499,0283,00aa,016a,006c,0.0,fc35,f666,fd7f,1.0,0.0,001e,0000007f,07ffffff
53,04/05/2021 21:59:27,18.900415,73.909383,0217.4,599.5,0,8,-45,0.0,14,3.607843,37.34,9.54,8.62,916.0,924.0,64.2,57.7,41.9,10bb,056d,04b0,0caf,0cb2,07fa,016c,04ad,0286,00aa,015e,006c,0.0,fc36,f666,fd7f,1.0,0.0,ffa7,000000ff,07ffffff
54,04/05/2021 21:59:28,18.900920,73.913392,0217.4,599.0,0,8,-45,0.0,14,3.764706,37.62,9.64,8.92,916.0,924.0,64.3,57.7,42.1,10bb,056b,04af,0caf,0cb2,07f8,016c,04a5,0295,00aa,0171,006c,0.0,fc37,f666,fd7f,1.0,0.0,003c,000000ff,07ffffff


Dealing with Oil Pressure

In [ ]:
OilPressure = []

for i in dataset['Oil Pressure (mBar)']:
  if 1 !=0:
    OilPressure.append(((int(i,16))*1.00)+(0))
  else:
    OilPressure.append(0)

In [ ]:
dataset.drop('Oil Pressure (mBar)',axis=1)
dataset['Oil Pressure (mBar)'] = (OilPressure)

In [ ]:
dataset.iloc[50:55]

,Time,Latitude,Longitude,Altitude,RPM,Driver Demand Torque (%),Engine Load (%),Engine Torque Mode,TPS (%),Percent Load Curret Speed,Fuel Rate (L-Hr),Vehicle Speed,Inj Q Cur (mg-st),Inj Q Tor (mg-st),Boost Pressure (mBar),Atmospheric Pressure (mBar),Coolant Temperature (*C),Oil Temperature (*C),Boost Temperature (*C),Oil Pressure (mBar),Battery Voltage (V),Cam Speed (rpm),Rail Pressure (mBar),Rail Pressure set (mBar),MU PWM (%),MU Vol (mm3-s),Torque Rat,Torque (Nm),TQ Limit Set,Main Injection (mg-st),Pilot Injection (mg-st),Pos 2 Injector (mg-st),EGR Prop (%),EGR Pos D (%),EGR Pos A (%),Clutch Switch,Brake Switch,Engine Grad (rpm),param1,param2
50,04/05/2021 21:59:24,18.900668,73.909515,0217.4,599.0,0,8,-61,0.0,15,3.764706,37.32,9.42,6.18,923.0,924.0,64.0,57.4,41.5,4303.0,0575,04b2,0bd3,0cb2,07f3,0176,04bc,02a0,00aa,0171,006d,0.0,fc35,f666,fd7f,1.0,0.0,ffc5,000001ff,07ffffff
51,04/05/2021 21:59:25,18.900592,73.909465,0217.4,600.0,0,8,-125,0.0,14,3.764706,37.37,8.88,8.14,917.0,924.0,64.2,57.5,41.7,4283.0,056d,04b0,0cdb,0cb2,07f6,0162,04a1,0293,00aa,0170,006c,0.0,fc35,f666,fd7f,1.0,0.0,003c,000000ff,07ffffff
52,04/05/2021 21:59:25,18.900592,73.909465,0217.4,602.0,0,8,-125,0.0,13,3.764706,37.37,8.88,8.14,917.0,924.0,64.3,57.6,41.8,4270.0,056c,04b0,0c99,0cb2,07f5,016e,0499,0283,00aa,016a,006c,0.0,fc35,f666,fd7f,1.0,0.0,001e,0000007f,07ffffff
53,04/05/2021 21:59:27,18.900415,73.909383,0217.4,599.5,0,8,-45,0.0,14,3.607843,37.34,9.54,8.62,916.0,924.0,64.2,57.7,41.9,4283.0,056d,04b0,0caf,0cb2,07fa,016c,04ad,0286,00aa,015e,006c,0.0,fc36,f666,fd7f,1.0,0.0,ffa7,000000ff,07ffffff
54,04/05/2021 21:59:28,18.900920,73.913392,0217.4,599.0,0,8,-45,0.0,14,3.764706,37.62,9.64,8.92,916.0,924.0,64.3,57.7,42.1,4283.0,056b,04af,0caf,0cb2,07f8,016c,04a5,0295,00aa,0171,006c,0.0,fc37,f666,fd7f,1.0,0.0,003c,000000ff,07ffffff


Dealing with Battery Voltage

In [ ]:
BVoltage = []

for i in dataset['Battery Voltage (V)']:
  if i != 0:
    BVoltage.append(((int(i,16))*0.02)+(0))
  else:
    BVoltage.append(0)

In [ ]:
dataset.drop('Battery Voltage (V)',axis=1)
dataset['Battery Voltage (V)'] = (BVoltage)

In [ ]:
dataset.iloc[50:55]

,Time,Latitude,Longitude,Altitude,RPM,Driver Demand Torque (%),Engine Load (%),Engine Torque Mode,TPS (%),Percent Load Curret Speed,Fuel Rate (L-Hr),Vehicle Speed,Inj Q Cur (mg-st),Inj Q Tor (mg-st),Boost Pressure (mBar),Atmospheric Pressure (mBar),Coolant Temperature (*C),Oil Temperature (*C),Boost Temperature (*C),Oil Pressure (mBar),Battery Voltage (V),Cam Speed (rpm),Rail Pressure (mBar),Rail Pressure set (mBar),MU PWM (%),MU Vol (mm3-s),Torque Rat,Torque (Nm),TQ Limit Set,Main Injection (mg-st),Pilot Injection (mg-st),Pos 2 Injector (mg-st),EGR Prop (%),EGR Pos D (%),EGR Pos A (%),Clutch Switch,Brake Switch,Engine Grad (rpm),param1,param2
50,04/05/2021 21:59:24,18.900668,73.909515,0217.4,599.0,0,8,-61,0.0,15,3.764706,37.32,9.42,6.18,923.0,924.0,64.0,57.4,41.5,4303.0,27.94,04b2,0bd3,0cb2,07f3,0176,04bc,02a0,00aa,0171,006d,0.0,fc35,f666,fd7f,1.0,0.0,ffc5,000001ff,07ffffff
51,04/05/2021 21:59:25,18.900592,73.909465,0217.4,600.0,0,8,-125,0.0,14,3.764706,37.37,8.88,8.14,917.0,924.0,64.2,57.5,41.7,4283.0,27.78,04b0,0cdb,0cb2,07f6,0162,04a1,0293,00aa,0170,006c,0.0,fc35,f666,fd7f,1.0,0.0,003c,000000ff,07ffffff
52,04/05/2021 21:59:25,18.900592,73.909465,0217.4,602.0,0,8,-125,0.0,13,3.764706,37.37,8.88,8.14,917.0,924.0,64.3,57.6,41.8,4270.0,27.76,04b0,0c99,0cb2,07f5,016e,0499,0283,00aa,016a,006c,0.0,fc35,f666,fd7f,1.0,0.0,001e,0000007f,07ffffff
53,04/05/2021 21:59:27,18.900415,73.909383,0217.4,599.5,0,8,-45,0.0,14,3.607843,37.34,9.54,8.62,916.0,924.0,64.2,57.7,41.9,4283.0,27.78,04b0,0caf,0cb2,07fa,016c,04ad,0286,00aa,015e,006c,0.0,fc36,f666,fd7f,1.0,0.0,ffa7,000000ff,07ffffff
54,04/05/2021 21:59:28,18.900920,73.913392,0217.4,599.0,0,8,-45,0.0,14,3.764706,37.62,9.64,8.92,916.0,924.0,64.3,57.7,42.1,4283.0,27.74,04af,0caf,0cb2,07f8,016c,04a5,0295,00aa,0171,006c,0.0,fc37,f666,fd7f,1.0,0.0,003c,000000ff,07ffffff


Dealing with Cam Speed

In [ ]:
CamSpeed = []

for i in dataset['Cam Speed (rpm)']:
  if i != 0:
    CamSpeed.append(((int(i,16))*0.50)+(0))
  else:
    CamSpeed.append(0)

In [ ]:
dataset.drop('Cam Speed (rpm)',axis=1)
dataset['Cam Speed (rpm)'] = (CamSpeed)

In [ ]:
dataset.iloc[50:55]

,Time,Latitude,Longitude,Altitude,RPM,Driver Demand Torque (%),Engine Load (%),Engine Torque Mode,TPS (%),Percent Load Curret Speed,Fuel Rate (L-Hr),Vehicle Speed,Inj Q Cur (mg-st),Inj Q Tor (mg-st),Boost Pressure (mBar),Atmospheric Pressure (mBar),Coolant Temperature (*C),Oil Temperature (*C),Boost Temperature (*C),Oil Pressure (mBar),Battery Voltage (V),Cam Speed (rpm),Rail Pressure (mBar),Rail Pressure set (mBar),MU PWM (%),MU Vol (mm3-s),Torque Rat,Torque (Nm),TQ Limit Set,Main Injection (mg-st),Pilot Injection (mg-st),Pos 2 Injector (mg-st),EGR Prop (%),EGR Pos D (%),EGR Pos A (%),Clutch Switch,Brake Switch,Engine Grad (rpm),param1,param2
50,04/05/2021 21:59:24,18.900668,73.909515,0217.4,599.0,0,8,-61,0.0,15,3.764706,37.32,9.42,6.18,923.0,924.0,64.0,57.4,41.5,4303.0,27.94,601.0,0bd3,0cb2,07f3,0176,04bc,02a0,00aa,0171,006d,0.0,fc35,f666,fd7f,1.0,0.0,ffc5,000001ff,07ffffff
51,04/05/2021 21:59:25,18.900592,73.909465,0217.4,600.0,0,8,-125,0.0,14,3.764706,37.37,8.88,8.14,917.0,924.0,64.2,57.5,41.7,4283.0,27.78,600.0,0cdb,0cb2,07f6,0162,04a1,0293,00aa,0170,006c,0.0,fc35,f666,fd7f,1.0,0.0,003c,000000ff,07ffffff
52,04/05/2021 21:59:25,18.900592,73.909465,0217.4,602.0,0,8,-125,0.0,13,3.764706,37.37,8.88,8.14,917.0,924.0,64.3,57.6,41.8,4270.0,27.76,600.0,0c99,0cb2,07f5,016e,0499,0283,00aa,016a,006c,0.0,fc35,f666,fd7f,1.0,0.0,001e,0000007f,07ffffff
53,04/05/2021 21:59:27,18.900415,73.909383,0217.4,599.5,0,8,-45,0.0,14,3.607843,37.34,9.54,8.62,916.0,924.0,64.2,57.7,41.9,4283.0,27.78,600.0,0caf,0cb2,07fa,016c,04ad,0286,00aa,015e,006c,0.0,fc36,f666,fd7f,1.0,0.0,ffa7,000000ff,07ffffff
54,04/05/2021 21:59:28,18.900920,73.913392,0217.4,599.0,0,8,-45,0.0,14,3.764706,37.62,9.64,8.92,916.0,924.0,64.3,57.7,42.1,4283.0,27.74,599.5,0caf,0cb2,07f8,016c,04a5,0295,00aa,0171,006c,0.0,fc37,f666,fd7f,1.0,0.0,003c,000000ff,07ffffff


Dealing with Rail Pressure

In [ ]:
RailPressure = []

for i in dataset['Rail Pressure (mBar)']:
  if i != 0:
    RailPressure.append(((int(i,16))*100)+(0))
  else:
    RailPressure.append(0)

In [ ]:
dataset.drop('Rail Pressure (mBar)',axis=1)
dataset['Rail Pressure (mBar)'] = (RailPressure)

In [ ]:
dataset.iloc[50:55]

,Time,Latitude,Longitude,Altitude,RPM,Driver Demand Torque (%),Engine Load (%),Engine Torque Mode,TPS (%),Percent Load Curret Speed,Fuel Rate (L-Hr),Vehicle Speed,Inj Q Cur (mg-st),Inj Q Tor (mg-st),Boost Pressure (mBar),Atmospheric Pressure (mBar),Coolant Temperature (*C),Oil Temperature (*C),Boost Temperature (*C),Oil Pressure (mBar),Battery Voltage (V),Cam Speed (rpm),Rail Pressure (mBar),Rail Pressure set (mBar),MU PWM (%),MU Vol (mm3-s),Torque Rat,Torque (Nm),TQ Limit Set,Main Injection (mg-st),Pilot Injection (mg-st),Pos 2 Injector (mg-st),EGR Prop (%),EGR Pos D (%),EGR Pos A (%),Clutch Switch,Brake Switch,Engine Grad (rpm),param1,param2
50,04/05/2021 21:59:24,18.900668,73.909515,0217.4,599.0,0,8,-61,0.0,15,3.764706,37.32,9.42,6.18,923.0,924.0,64.0,57.4,41.5,4303.0,27.94,601.0,302700,0cb2,07f3,0176,04bc,02a0,00aa,0171,006d,0.0,fc35,f666,fd7f,1.0,0.0,ffc5,000001ff,07ffffff
51,04/05/2021 21:59:25,18.900592,73.909465,0217.4,600.0,0,8,-125,0.0,14,3.764706,37.37,8.88,8.14,917.0,924.0,64.2,57.5,41.7,4283.0,27.78,600.0,329100,0cb2,07f6,0162,04a1,0293,00aa,0170,006c,0.0,fc35,f666,fd7f,1.0,0.0,003c,000000ff,07ffffff
52,04/05/2021 21:59:25,18.900592,73.909465,0217.4,602.0,0,8,-125,0.0,13,3.764706,37.37,8.88,8.14,917.0,924.0,64.3,57.6,41.8,4270.0,27.76,600.0,322500,0cb2,07f5,016e,0499,0283,00aa,016a,006c,0.0,fc35,f666,fd7f,1.0,0.0,001e,0000007f,07ffffff
53,04/05/2021 21:59:27,18.900415,73.909383,0217.4,599.5,0,8,-45,0.0,14,3.607843,37.34,9.54,8.62,916.0,924.0,64.2,57.7,41.9,4283.0,27.78,600.0,324700,0cb2,07fa,016c,04ad,0286,00aa,015e,006c,0.0,fc36,f666,fd7f,1.0,0.0,ffa7,000000ff,07ffffff
54,04/05/2021 21:59:28,18.900920,73.913392,0217.4,599.0,0,8,-45,0.0,14,3.764706,37.62,9.64,8.92,916.0,924.0,64.3,57.7,42.1,4283.0,27.74,599.5,324700,0cb2,07f8,016c,04a5,0295,00aa,0171,006c,0.0,fc37,f666,fd7f,1.0,0.0,003c,000000ff,07ffffff


Dealing with Rail Pressure Set 

In [ ]:
RailPressureSet = []

for i in dataset['Rail Pressure set (mBar)']:
  if i != 0:
    RailPressureSet.append(((int(i,16))*100)+(0))
  else:
    RailPressure.append(0)

In [ ]:
dataset.drop('Rail Pressure set (mBar)',axis=1)
dataset['Rail Pressure set (mBar)'] = (RailPressureSet)

In [ ]:
dataset.iloc[50:55]

,Time,Latitude,Longitude,Altitude,RPM,Driver Demand Torque (%),Engine Load (%),Engine Torque Mode,TPS (%),Percent Load Curret Speed,Fuel Rate (L-Hr),Vehicle Speed,Inj Q Cur (mg-st),Inj Q Tor (mg-st),Boost Pressure (mBar),Atmospheric Pressure (mBar),Coolant Temperature (*C),Oil Temperature (*C),Boost Temperature (*C),Oil Pressure (mBar),Battery Voltage (V),Cam Speed (rpm),Rail Pressure (mBar),Rail Pressure set (mBar),MU PWM (%),MU Vol (mm3-s),Torque Rat,Torque (Nm),TQ Limit Set,Main Injection (mg-st),Pilot Injection (mg-st),Pos 2 Injector (mg-st),EGR Prop (%),EGR Pos D (%),EGR Pos A (%),Clutch Switch,Brake Switch,Engine Grad (rpm),param1,param2
50,04/05/2021 21:59:24,18.900668,73.909515,0217.4,599.0,0,8,-61,0.0,15,3.764706,37.32,9.42,6.18,923.0,924.0,64.0,57.4,41.5,4303.0,27.94,601.0,302700,325000,07f3,0176,04bc,02a0,00aa,0171,006d,0.0,fc35,f666,fd7f,1.0,0.0,ffc5,000001ff,07ffffff
51,04/05/2021 21:59:25,18.900592,73.909465,0217.4,600.0,0,8,-125,0.0,14,3.764706,37.37,8.88,8.14,917.0,924.0,64.2,57.5,41.7,4283.0,27.78,600.0,329100,325000,07f6,0162,04a1,0293,00aa,0170,006c,0.0,fc35,f666,fd7f,1.0,0.0,003c,000000ff,07ffffff
52,04/05/2021 21:59:25,18.900592,73.909465,0217.4,602.0,0,8,-125,0.0,13,3.764706,37.37,8.88,8.14,917.0,924.0,64.3,57.6,41.8,4270.0,27.76,600.0,322500,325000,07f5,016e,0499,0283,00aa,016a,006c,0.0,fc35,f666,fd7f,1.0,0.0,001e,0000007f,07ffffff
53,04/05/2021 21:59:27,18.900415,73.909383,0217.4,599.5,0,8,-45,0.0,14,3.607843,37.34,9.54,8.62,916.0,924.0,64.2,57.7,41.9,4283.0,27.78,600.0,324700,325000,07fa,016c,04ad,0286,00aa,015e,006c,0.0,fc36,f666,fd7f,1.0,0.0,ffa7,000000ff,07ffffff
54,04/05/2021 21:59:28,18.900920,73.913392,0217.4,599.0,0,8,-45,0.0,14,3.764706,37.62,9.64,8.92,916.0,924.0,64.3,57.7,42.1,4283.0,27.74,599.5,324700,325000,07f8,016c,04a5,0295,00aa,0171,006c,0.0,fc37,f666,fd7f,1.0,0.0,003c,000000ff,07ffffff


Dealing with MU Vol %

In [ ]:
MUPWM = []

for i in dataset['MU PWM (%)']:
  if i != 0:
    MUPWM.append(((int(i,16))*0.01)+(0))
  else:
    MUPWM.append(0)

In [ ]:
dataset.drop('MU PWM (%)',axis=1)
dataset['MU PWM (%)'] = (MUPWM)

In [ ]:
dataset.iloc[50:55]

,Time,Latitude,Longitude,Altitude,RPM,Driver Demand Torque (%),Engine Load (%),Engine Torque Mode,TPS (%),Percent Load Curret Speed,Fuel Rate (L-Hr),Vehicle Speed,Inj Q Cur (mg-st),Inj Q Tor (mg-st),Boost Pressure (mBar),Atmospheric Pressure (mBar),Coolant Temperature (*C),Oil Temperature (*C),Boost Temperature (*C),Oil Pressure (mBar),Battery Voltage (V),Cam Speed (rpm),Rail Pressure (mBar),Rail Pressure set (mBar),MU PWM (%),MU Vol (mm3-s),Torque Rat,Torque (Nm),TQ Limit Set,Main Injection (mg-st),Pilot Injection (mg-st),Pos 2 Injector (mg-st),EGR Prop (%),EGR Pos D (%),EGR Pos A (%),Clutch Switch,Brake Switch,Engine Grad (rpm),param1,param2
50,04/05/2021 21:59:24,18.900668,73.909515,0217.4,599.0,0,8,-61,0.0,15,3.764706,37.32,9.42,6.18,923.0,924.0,64.0,57.4,41.5,4303.0,27.94,601.0,302700,325000,20.35,0176,04bc,02a0,00aa,0171,006d,0.0,fc35,f666,fd7f,1.0,0.0,ffc5,000001ff,07ffffff
51,04/05/2021 21:59:25,18.900592,73.909465,0217.4,600.0,0,8,-125,0.0,14,3.764706,37.37,8.88,8.14,917.0,924.0,64.2,57.5,41.7,4283.0,27.78,600.0,329100,325000,20.38,0162,04a1,0293,00aa,0170,006c,0.0,fc35,f666,fd7f,1.0,0.0,003c,000000ff,07ffffff
52,04/05/2021 21:59:25,18.900592,73.909465,0217.4,602.0,0,8,-125,0.0,13,3.764706,37.37,8.88,8.14,917.0,924.0,64.3,57.6,41.8,4270.0,27.76,600.0,322500,325000,20.37,016e,0499,0283,00aa,016a,006c,0.0,fc35,f666,fd7f,1.0,0.0,001e,0000007f,07ffffff
53,04/05/2021 21:59:27,18.900415,73.909383,0217.4,599.5,0,8,-45,0.0,14,3.607843,37.34,9.54,8.62,916.0,924.0,64.2,57.7,41.9,4283.0,27.78,600.0,324700,325000,20.42,016c,04ad,0286,00aa,015e,006c,0.0,fc36,f666,fd7f,1.0,0.0,ffa7,000000ff,07ffffff
54,04/05/2021 21:59:28,18.900920,73.913392,0217.4,599.0,0,8,-45,0.0,14,3.764706,37.62,9.64,8.92,916.0,924.0,64.3,57.7,42.1,4283.0,27.74,599.5,324700,325000,20.40,016c,04a5,0295,00aa,0171,006c,0.0,fc37,f666,fd7f,1.0,0.0,003c,000000ff,07ffffff


Dealing with MU Vol (mm3-s)

In [ ]:
MUVol = []

for i in dataset['MU Vol (mm3-s)']:
  if i != 0:
    MUVol.append(((int(i,16))*10)+(0))
  else:
    MUVol.append(0)

In [ ]:
dataset.drop('MU Vol (mm3-s)',axis=1)
dataset['MU Vol (mm3-s)'] = (MUVol)

In [ ]:
dataset.iloc[50:55]

,Time,Latitude,Longitude,Altitude,RPM,Driver Demand Torque (%),Engine Load (%),Engine Torque Mode,TPS (%),Percent Load Curret Speed,Fuel Rate (L-Hr),Vehicle Speed,Inj Q Cur (mg-st),Inj Q Tor (mg-st),Boost Pressure (mBar),Atmospheric Pressure (mBar),Coolant Temperature (*C),Oil Temperature (*C),Boost Temperature (*C),Oil Pressure (mBar),Battery Voltage (V),Cam Speed (rpm),Rail Pressure (mBar),Rail Pressure set (mBar),MU PWM (%),MU Vol (mm3-s),Torque Rat,Torque (Nm),TQ Limit Set,Main Injection (mg-st),Pilot Injection (mg-st),Pos 2 Injector (mg-st),EGR Prop (%),EGR Pos D (%),EGR Pos A (%),Clutch Switch,Brake Switch,Engine Grad (rpm),param1,param2
50,04/05/2021 21:59:24,18.900668,73.909515,0217.4,599.0,0,8,-61,0.0,15,3.764706,37.32,9.42,6.18,923.0,924.0,64.0,57.4,41.5,4303.0,27.94,601.0,302700,325000,20.35,3740,04bc,02a0,00aa,0171,006d,0.0,fc35,f666,fd7f,1.0,0.0,ffc5,000001ff,07ffffff
51,04/05/2021 21:59:25,18.900592,73.909465,0217.4,600.0,0,8,-125,0.0,14,3.764706,37.37,8.88,8.14,917.0,924.0,64.2,57.5,41.7,4283.0,27.78,600.0,329100,325000,20.38,3540,04a1,0293,00aa,0170,006c,0.0,fc35,f666,fd7f,1.0,0.0,003c,000000ff,07ffffff
52,04/05/2021 21:59:25,18.900592,73.909465,0217.4,602.0,0,8,-125,0.0,13,3.764706,37.37,8.88,8.14,917.0,924.0,64.3,57.6,41.8,4270.0,27.76,600.0,322500,325000,20.37,3660,0499,0283,00aa,016a,006c,0.0,fc35,f666,fd7f,1.0,0.0,001e,0000007f,07ffffff
53,04/05/2021 21:59:27,18.900415,73.909383,0217.4,599.5,0,8,-45,0.0,14,3.607843,37.34,9.54,8.62,916.0,924.0,64.2,57.7,41.9,4283.0,27.78,600.0,324700,325000,20.42,3640,04ad,0286,00aa,015e,006c,0.0,fc36,f666,fd7f,1.0,0.0,ffa7,000000ff,07ffffff
54,04/05/2021 21:59:28,18.900920,73.913392,0217.4,599.0,0,8,-45,0.0,14,3.764706,37.62,9.64,8.92,916.0,924.0,64.3,57.7,42.1,4283.0,27.74,599.5,324700,325000,20.40,3640,04a5,0295,00aa,0171,006c,0.0,fc37,f666,fd7f,1.0,0.0,003c,000000ff,07ffffff


Dealing with Torque Rat

In [ ]:
TorqueRat = []

for i in dataset['Torque Rat']:
  if i != 0:
    TorqueRat.append(((int(i,16))*0.01)+(0))
  else:
    TorqueRat.append(0)

In [ ]:
dataset.drop('Torque Rat',axis=1)
dataset['Torque Rat'] = (TorqueRat)

In [ ]:
dataset.iloc[50:55]

,Time,Latitude,Longitude,Altitude,RPM,Driver Demand Torque (%),Engine Load (%),Engine Torque Mode,TPS (%),Percent Load Curret Speed,Fuel Rate (L-Hr),Vehicle Speed,Inj Q Cur (mg-st),Inj Q Tor (mg-st),Boost Pressure (mBar),Atmospheric Pressure (mBar),Coolant Temperature (*C),Oil Temperature (*C),Boost Temperature (*C),Oil Pressure (mBar),Battery Voltage (V),Cam Speed (rpm),Rail Pressure (mBar),Rail Pressure set (mBar),MU PWM (%),MU Vol (mm3-s),Torque Rat,Torque (Nm),TQ Limit Set,Main Injection (mg-st),Pilot Injection (mg-st),Pos 2 Injector (mg-st),EGR Prop (%),EGR Pos D (%),EGR Pos A (%),Clutch Switch,Brake Switch,Engine Grad (rpm),param1,param2
50,04/05/2021 21:59:24,18.900668,73.909515,0217.4,599.0,0,8,-61,0.0,15,3.764706,37.32,9.42,6.18,923.0,924.0,64.0,57.4,41.5,4303.0,27.94,601.0,302700,325000,20.35,3740,12.12,02a0,00aa,0171,006d,0.0,fc35,f666,fd7f,1.0,0.0,ffc5,000001ff,07ffffff
51,04/05/2021 21:59:25,18.900592,73.909465,0217.4,600.0,0,8,-125,0.0,14,3.764706,37.37,8.88,8.14,917.0,924.0,64.2,57.5,41.7,4283.0,27.78,600.0,329100,325000,20.38,3540,11.85,0293,00aa,0170,006c,0.0,fc35,f666,fd7f,1.0,0.0,003c,000000ff,07ffffff
52,04/05/2021 21:59:25,18.900592,73.909465,0217.4,602.0,0,8,-125,0.0,13,3.764706,37.37,8.88,8.14,917.0,924.0,64.3,57.6,41.8,4270.0,27.76,600.0,322500,325000,20.37,3660,11.77,0283,00aa,016a,006c,0.0,fc35,f666,fd7f,1.0,0.0,001e,0000007f,07ffffff
53,04/05/2021 21:59:27,18.900415,73.909383,0217.4,599.5,0,8,-45,0.0,14,3.607843,37.34,9.54,8.62,916.0,924.0,64.2,57.7,41.9,4283.0,27.78,600.0,324700,325000,20.42,3640,11.97,0286,00aa,015e,006c,0.0,fc36,f666,fd7f,1.0,0.0,ffa7,000000ff,07ffffff
54,04/05/2021 21:59:28,18.900920,73.913392,0217.4,599.0,0,8,-45,0.0,14,3.764706,37.62,9.64,8.92,916.0,924.0,64.3,57.7,42.1,4283.0,27.74,599.5,324700,325000,20.40,3640,11.89,0295,00aa,0171,006c,0.0,fc37,f666,fd7f,1.0,0.0,003c,000000ff,07ffffff


Dealing with Torque

In [ ]:
Torque = []

for i in dataset['Torque (Nm)']:
  if i != 0:
    Torque.append(((int(i,16))*0.10)+(0))
  else:
    Torque.append(0)

In [ ]:
dataset.drop('Torque (Nm)',axis=1)
dataset['Torque (Nm)'] = (Torque)

In [ ]:
dataset.iloc[50:55]

,Time,Latitude,Longitude,Altitude,RPM,Driver Demand Torque (%),Engine Load (%),Engine Torque Mode,TPS (%),Percent Load Curret Speed,Fuel Rate (L-Hr),Vehicle Speed,Inj Q Cur (mg-st),Inj Q Tor (mg-st),Boost Pressure (mBar),Atmospheric Pressure (mBar),Coolant Temperature (*C),Oil Temperature (*C),Boost Temperature (*C),Oil Pressure (mBar),Battery Voltage (V),Cam Speed (rpm),Rail Pressure (mBar),Rail Pressure set (mBar),MU PWM (%),MU Vol (mm3-s),Torque Rat,Torque (Nm),TQ Limit Set,Main Injection (mg-st),Pilot Injection (mg-st),Pos 2 Injector (mg-st),EGR Prop (%),EGR Pos D (%),EGR Pos A (%),Clutch Switch,Brake Switch,Engine Grad (rpm),param1,param2
50,04/05/2021 21:59:24,18.900668,73.909515,0217.4,599.0,0,8,-61,0.0,15,3.764706,37.32,9.42,6.18,923.0,924.0,64.0,57.4,41.5,4303.0,27.94,601.0,302700,325000,20.35,3740,12.12,67.2,00aa,0171,006d,0.0,fc35,f666,fd7f,1.0,0.0,ffc5,000001ff,07ffffff
51,04/05/2021 21:59:25,18.900592,73.909465,0217.4,600.0,0,8,-125,0.0,14,3.764706,37.37,8.88,8.14,917.0,924.0,64.2,57.5,41.7,4283.0,27.78,600.0,329100,325000,20.38,3540,11.85,65.9,00aa,0170,006c,0.0,fc35,f666,fd7f,1.0,0.0,003c,000000ff,07ffffff
52,04/05/2021 21:59:25,18.900592,73.909465,0217.4,602.0,0,8,-125,0.0,13,3.764706,37.37,8.88,8.14,917.0,924.0,64.3,57.6,41.8,4270.0,27.76,600.0,322500,325000,20.37,3660,11.77,64.3,00aa,016a,006c,0.0,fc35,f666,fd7f,1.0,0.0,001e,0000007f,07ffffff
53,04/05/2021 21:59:27,18.900415,73.909383,0217.4,599.5,0,8,-45,0.0,14,3.607843,37.34,9.54,8.62,916.0,924.0,64.2,57.7,41.9,4283.0,27.78,600.0,324700,325000,20.42,3640,11.97,64.6,00aa,015e,006c,0.0,fc36,f666,fd7f,1.0,0.0,ffa7,000000ff,07ffffff
54,04/05/2021 21:59:28,18.900920,73.913392,0217.4,599.0,0,8,-45,0.0,14,3.764706,37.62,9.64,8.92,916.0,924.0,64.3,57.7,42.1,4283.0,27.74,599.5,324700,325000,20.40,3640,11.89,66.1,00aa,0171,006c,0.0,fc37,f666,fd7f,1.0,0.0,003c,000000ff,07ffffff


Dealing with TQ Limit Set

In [ ]:
TQLimit = []

for i in dataset['TQ Limit Set']:
  if i != 0:
    TQLimit.append(((int(i,16))*0.10)+(0))
  else:
    TQLimit.append(0)

In [ ]:
dataset.drop('TQ Limit Set',axis=1)
dataset['TQ Limit Set'] = (TQLimit)

In [ ]:
dataset.head()

,Time,Latitude,Longitude,Altitude,RPM,Driver Demand Torque (%),Engine Load (%),Engine Torque Mode,TPS (%),Percent Load Curret Speed,Fuel Rate (L-Hr),Vehicle Speed,Inj Q Cur (mg-st),Inj Q Tor (mg-st),Boost Pressure (mBar),Atmospheric Pressure (mBar),Coolant Temperature (*C),Oil Temperature (*C),Boost Temperature (*C),Oil Pressure (mBar),Battery Voltage (V),Cam Speed (rpm),Rail Pressure (mBar),Rail Pressure set (mBar),MU PWM (%),MU Vol (mm3-s),Torque Rat,Torque (Nm),TQ Limit Set,Main Injection (mg-st),Pilot Injection (mg-st),Pos 2 Injector (mg-st),EGR Prop (%),EGR Pos D (%),EGR Pos A (%),Clutch Switch,Brake Switch,Engine Grad (rpm),param1,param2
0,04/05/2021 29:29:45,0.0,0.0,0,554.0,0,8,-29,0.0,14,3.450980,0.0,9.58,9.04,914.0,924.0,64.6,55.3,54.0,4405.0,27.24,551.5,320400,325000,20.74,3300,11.93,65.9,17.0,017b,006c,0.0,fc1f,f666,fd64,1.0,0.0,001b,000000ff,07ffffff
1,04/05/2021 29:29:46,0.0,0.0,0,550.0,0,8,-29,0.0,15,3.607843,0.0,9.90,9.32,913.0,924.0,64.6,55.3,53.2,4391.0,27.36,550.0,322500,325000,20.76,3190,12.30,66.2,17.0,0177,006c,0.0,fc28,f666,fd64,1.0,0.0,0000,000000ff,07ffffff
2,04/05/2021 29:29:47,0.0,0.0,0,550.5,0,8,-45,0.0,14,3.450980,0.0,9.62,9.48,912.0,924.0,64.5,55.3,52.6,4344.0,27.70,550.0,324700,325000,20.50,3290,12.58,66.8,17.0,017c,006c,0.0,fc2f,f666,fd64,1.0,0.0,0000,000000ff,07ffffff
3,04/05/2021 29:29:48,0.0,0.0,0,550.0,0,8,-29,0.0,15,3.607843,0.0,9.70,9.56,912.0,924.0,64.4,55.4,52.1,4371.0,27.84,550.0,324700,325000,20.31,3330,12.17,66.4,17.0,0179,006c,0.0,fc38,f666,fd64,1.0,0.0,0000,000001ff,07ffffff
4,04/05/2021 29:29:49,0.0,0.0,0,550.0,0,8,-45,0.0,14,3.450980,0.0,9.56,9.58,911.0,923.0,64.3,55.4,51.6,4303.0,27.92,550.5,324700,325000,20.33,3340,12.02,64.5,17.0,0164,006c,0.0,fc38,f666,fd64,1.0,0.0,ffe5,000000ff,07ffffff


Dealing with Main Injection (mg-st)

In [ ]:
MInjection = []

for i in dataset['Main Injection (mg-st)']:
  if i != 0:
    MInjection.append(((int(i,16))*0.02)+(0))
  else:
    MInjection.append(0)

In [ ]:
dataset.drop('Main Injection (mg-st)',axis=1)
dataset['Main Injection (mg-st)'] = (MInjection)

In [ ]:
dataset.head()

,Time,Latitude,Longitude,Altitude,RPM,Driver Demand Torque (%),Engine Load (%),Engine Torque Mode,TPS (%),Percent Load Curret Speed,Fuel Rate (L-Hr),Vehicle Speed,Inj Q Cur (mg-st),Inj Q Tor (mg-st),Boost Pressure (mBar),Atmospheric Pressure (mBar),Coolant Temperature (*C),Oil Temperature (*C),Boost Temperature (*C),Oil Pressure (mBar),Battery Voltage (V),Cam Speed (rpm),Rail Pressure (mBar),Rail Pressure set (mBar),MU PWM (%),MU Vol (mm3-s),Torque Rat,Torque (Nm),TQ Limit Set,Main Injection (mg-st),Pilot Injection (mg-st),Pos 2 Injector (mg-st),EGR Prop (%),EGR Pos D (%),EGR Pos A (%),Clutch Switch,Brake Switch,Engine Grad (rpm),param1,param2
0,04/05/2021 29:29:45,0.0,0.0,0,554.0,0,8,-29,0.0,14,3.450980,0.0,9.58,9.04,914.0,924.0,64.6,55.3,54.0,4405.0,27.24,551.5,320400,325000,20.74,3300,11.93,65.9,17.0,7.58,006c,0.0,fc1f,f666,fd64,1.0,0.0,001b,000000ff,07ffffff
1,04/05/2021 29:29:46,0.0,0.0,0,550.0,0,8,-29,0.0,15,3.607843,0.0,9.90,9.32,913.0,924.0,64.6,55.3,53.2,4391.0,27.36,550.0,322500,325000,20.76,3190,12.30,66.2,17.0,7.50,006c,0.0,fc28,f666,fd64,1.0,0.0,0000,000000ff,07ffffff
2,04/05/2021 29:29:47,0.0,0.0,0,550.5,0,8,-45,0.0,14,3.450980,0.0,9.62,9.48,912.0,924.0,64.5,55.3,52.6,4344.0,27.70,550.0,324700,325000,20.50,3290,12.58,66.8,17.0,7.60,006c,0.0,fc2f,f666,fd64,1.0,0.0,0000,000000ff,07ffffff
3,04/05/2021 29:29:48,0.0,0.0,0,550.0,0,8,-29,0.0,15,3.607843,0.0,9.70,9.56,912.0,924.0,64.4,55.4,52.1,4371.0,27.84,550.0,324700,325000,20.31,3330,12.17,66.4,17.0,7.54,006c,0.0,fc38,f666,fd64,1.0,0.0,0000,000001ff,07ffffff
4,04/05/2021 29:29:49,0.0,0.0,0,550.0,0,8,-45,0.0,14,3.450980,0.0,9.56,9.58,911.0,923.0,64.3,55.4,51.6,4303.0,27.92,550.5,324700,325000,20.33,3340,12.02,64.5,17.0,7.12,006c,0.0,fc38,f666,fd64,1.0,0.0,ffe5,000000ff,07ffffff


Dealing with Pilot Injection (mg-st)

In [ ]:
PInjection = []

for i in dataset['Pilot Injection (mg-st)']:
  if i != 0:
    PInjection.append(((int(i))*0.02)+(0))
  else:
    PInjection.append(0)

In [ ]:
dataset.drop('Pilot Injection (mg-st)',axis=1)
dataset['Pilot Injection (mg-st)'] = (PInjection)

In [ ]:
dataset.head()

,Time,Latitude,Longitude,Altitude,RPM,Driver Demand Torque (%),Engine Load (%),Engine Torque Mode,TPS (%),Percent Load Curret Speed,Fuel Rate (L-Hr),Vehicle Speed,Inj Q Cur (mg-st),Inj Q Tor (mg-st),Boost Pressure (mBar),Atmospheric Pressure (mBar),Coolant Temperature (*C),Oil Temperature (*C),Boost Temperature (*C),Oil Pressure (mBar),Battery Voltage (V),Cam Speed (rpm),Rail Pressure (mBar),Rail Pressure set (mBar),MU PWM (%),MU Vol (mm3-s),Torque Rat,Torque (Nm),TQ Limit Set,Main Injection (mg-st),Pilot Injection (mg-st),Pos 2 Injector (mg-st),EGR Prop (%),EGR Pos D (%),EGR Pos A (%),Clutch Switch,Brake Switch,Engine Grad (rpm),param1,param2
0,235945,0,0,0,d412,7d,86,60,00,11,1c00,00a4,021b,0168,03a9,039f,0e16,0ddd,0cee,0ca2,051e,04b4,0cdb,0cb2,089e,019a,057f,73.5,17.0,9.06,1.28,0.0,fbf5,f666,fd81,1.0,0.0,001e,000000ff,07ffffff
1,235946,0,0,0,b412,7d,87,50,00,12,1d00,0111,024a,01b5,03a7,03a0,0e0e,0ddd,0cea,0c9b,0519,04ae,0caf,0cb2,0898,01a1,05dc,77.6,17.0,9.18,1.28,0.0,fbfb,f666,fd81,1.0,0.0,ffa7,000000ff,07ffffff
2,235947,0,0,0,6411,7d,98,70,00,34,4100,0158,053f,02de,03a7,039f,0e07,0ddd,0ce4,0bed,0517,045a,0caf,0fe3,072b,04fb,178b,345.4,17.0,36.44,1.28,0.0,fbf3,f666,fd81,1.0,0.0,02de,000000ff,07ffffff
3,235948,0,0,0,f413,90,a0,51,1e,3e,5500,0201,05e2,05ae,03b1,03a0,0dff,0ddd,0ce0,0cd1,0526,04fb,1194,11b1,081d,027a,15f5,303.6,17.0,30.94,1.28,0.0,fc00,f666,fd81,1.0,0.0,ffc2,000001ff,07ffffff
4,235949,0,0,0,3413,98,a2,71,26,56,6700,030f,07fd,0737,03bf,03a0,0df8,0ddd,0cdc,0caf,0533,04c2,15e1,153f,07ad,0348,0e4c,192.4,17.0,20.80,1.28,0.0,fc0d,f666,fd81,1.0,0.0,ffdf,000000ff,07ffffff


Dealing with EGR Prop (%)

In [ ]:
EGRProp = []

for i in dataset['EGR Prop (%)']:
  if i != 0:
    EGRProp.append(((int(i,16))*0.01)+(0))
  else:
    EGRProp.append(0)

In [ ]:
dataset.drop('EGR Prop (%)',axis=1)
dataset['EGR Prop (%)'] = (EGRProp)

In [ ]:
dataset.head()

,Time,Latitude,Longitude,Altitude,RPM,Driver Demand Torque (%),Engine Load (%),Engine Torque Mode,TPS (%),Percent Load Curret Speed,Fuel Rate (L-Hr),Vehicle Speed,Inj Q Cur (mg-st),Inj Q Tor (mg-st),Boost Pressure (mBar),Atmospheric Pressure (mBar),Coolant Temperature (*C),Oil Temperature (*C),Boost Temperature (*C),Oil Pressure (mBar),Battery Voltage (V),Cam Speed (rpm),Rail Pressure (mBar),Rail Pressure set (mBar),MU PWM (%),MU Vol (mm3-s),Torque Rat,Torque (Nm),TQ Limit Set,Main Injection (mg-st),Pilot Injection (mg-st),Pos 2 Injector (mg-st),EGR Prop (%),EGR Pos D (%),EGR Pos A (%),Clutch Switch,Brake Switch,Engine Grad (rpm),param1,param2
0,04/05/2021 29:29:45,0.0,0.0,0,554.0,0,8,-29,0.0,14,3.450980,0.0,9.58,9.04,914.0,924.0,64.6,55.3,54.0,4405.0,27.24,551.5,320400,325000,20.74,3300,11.93,65.9,17.0,7.58,006c,0.0,645.43,f666,fd64,1.0,0.0,001b,000000ff,07ffffff
1,04/05/2021 29:29:46,0.0,0.0,0,550.0,0,8,-29,0.0,15,3.607843,0.0,9.90,9.32,913.0,924.0,64.6,55.3,53.2,4391.0,27.36,550.0,322500,325000,20.76,3190,12.30,66.2,17.0,7.50,006c,0.0,645.52,f666,fd64,1.0,0.0,0000,000000ff,07ffffff
2,04/05/2021 29:29:47,0.0,0.0,0,550.5,0,8,-45,0.0,14,3.450980,0.0,9.62,9.48,912.0,924.0,64.5,55.3,52.6,4344.0,27.70,550.0,324700,325000,20.50,3290,12.58,66.8,17.0,7.60,006c,0.0,645.59,f666,fd64,1.0,0.0,0000,000000ff,07ffffff
3,04/05/2021 29:29:48,0.0,0.0,0,550.0,0,8,-29,0.0,15,3.607843,0.0,9.70,9.56,912.0,924.0,64.4,55.4,52.1,4371.0,27.84,550.0,324700,325000,20.31,3330,12.17,66.4,17.0,7.54,006c,0.0,645.68,f666,fd64,1.0,0.0,0000,000001ff,07ffffff
4,04/05/2021 29:29:49,0.0,0.0,0,550.0,0,8,-45,0.0,14,3.450980,0.0,9.56,9.58,911.0,923.0,64.3,55.4,51.6,4303.0,27.92,550.5,324700,325000,20.33,3340,12.02,64.5,17.0,7.12,006c,0.0,645.68,f666,fd64,1.0,0.0,ffe5,000000ff,07ffffff


Dealing with EGR Pos D (%)

In [ ]:
EGRPosD = []

for i in dataset['EGR Pos D (%)']:
  if i != 0:
    EGRPosD.append(((int(i,16))*0.01)+(0))
  else:
    EGRPosD.append(0)

In [ ]:
dataset.drop('EGR Pos D (%)',axis=1)
dataset['EGR Pos D (%)'] = (EGRPosD)

In [ ]:
dataset.head()

,Time,Latitude,Longitude,Altitude,RPM,Driver Demand Torque (%),Engine Load (%),Engine Torque Mode,TPS (%),Percent Load Curret Speed,Fuel Rate (L-Hr),Vehicle Speed,Inj Q Cur (mg-st),Inj Q Tor (mg-st),Boost Pressure (mBar),Atmospheric Pressure (mBar),Coolant Temperature (*C),Oil Temperature (*C),Boost Temperature (*C),Oil Pressure (mBar),Battery Voltage (V),Cam Speed (rpm),Rail Pressure (mBar),Rail Pressure set (mBar),MU PWM (%),MU Vol (mm3-s),Torque Rat,Torque (Nm),TQ Limit Set,Main Injection (mg-st),Pilot Injection (mg-st),Pos 2 Injector (mg-st),EGR Prop (%),EGR Pos D (%),EGR Pos A (%),Clutch Switch,Brake Switch,Engine Grad (rpm),param1,param2
0,04/05/2021 29:29:45,0.0,0.0,0,554.0,0,8,-29,0.0,14,3.450980,0.0,9.58,9.04,914.0,924.0,64.6,55.3,54.0,4405.0,27.24,551.5,320400,325000,20.74,3300,11.93,65.9,17.0,7.58,006c,0.0,645.43,630.78,fd64,1.0,0.0,001b,000000ff,07ffffff
1,04/05/2021 29:29:46,0.0,0.0,0,550.0,0,8,-29,0.0,15,3.607843,0.0,9.90,9.32,913.0,924.0,64.6,55.3,53.2,4391.0,27.36,550.0,322500,325000,20.76,3190,12.30,66.2,17.0,7.50,006c,0.0,645.52,630.78,fd64,1.0,0.0,0000,000000ff,07ffffff
2,04/05/2021 29:29:47,0.0,0.0,0,550.5,0,8,-45,0.0,14,3.450980,0.0,9.62,9.48,912.0,924.0,64.5,55.3,52.6,4344.0,27.70,550.0,324700,325000,20.50,3290,12.58,66.8,17.0,7.60,006c,0.0,645.59,630.78,fd64,1.0,0.0,0000,000000ff,07ffffff
3,04/05/2021 29:29:48,0.0,0.0,0,550.0,0,8,-29,0.0,15,3.607843,0.0,9.70,9.56,912.0,924.0,64.4,55.4,52.1,4371.0,27.84,550.0,324700,325000,20.31,3330,12.17,66.4,17.0,7.54,006c,0.0,645.68,630.78,fd64,1.0,0.0,0000,000001ff,07ffffff
4,04/05/2021 29:29:49,0.0,0.0,0,550.0,0,8,-45,0.0,14,3.450980,0.0,9.56,9.58,911.0,923.0,64.3,55.4,51.6,4303.0,27.92,550.5,324700,325000,20.33,3340,12.02,64.5,17.0,7.12,006c,0.0,645.68,630.78,fd64,1.0,0.0,ffe5,000000ff,07ffffff


Dealing with EGR Pos A (%)

In [ ]:
EGRPosA = []

for i in dataset['EGR Pos A (%)']:
  if i != 0:
    EGRPosA.append(((int(i,16))*0.01)+(0))
  else:
    EGRPosA.append(0)

In [ ]:
dataset.drop('EGR Pos A (%)',axis=1)
dataset['EGR Pos A (%)'] = (EGRPosA)

In [ ]:
dataset.head()

,Time,Latitude,Longitude,Altitude,RPM,Driver Demand Torque (%),Engine Load (%),Engine Torque Mode,TPS (%),Percent Load Curret Speed,Fuel Rate (L-Hr),Vehicle Speed,Inj Q Cur (mg-st),Inj Q Tor (mg-st),Boost Pressure (mBar),Atmospheric Pressure (mBar),Coolant Temperature (*C),Oil Temperature (*C),Boost Temperature (*C),Oil Pressure (mBar),Battery Voltage (V),Cam Speed (rpm),Rail Pressure (mBar),Rail Pressure set (mBar),MU PWM (%),MU Vol (mm3-s),Torque Rat,Torque (Nm),TQ Limit Set,Main Injection (mg-st),Pilot Injection (mg-st),Pos 2 Injector (mg-st),EGR Prop (%),EGR Pos D (%),EGR Pos A (%),Clutch Switch,Brake Switch,Engine Grad (rpm),param1,param2
0,04/05/2021 29:29:45,0.0,0.0,0,554.0,0,8,-29,0.0,14,3.450980,0.0,9.58,9.04,914.0,924.0,64.6,55.3,54.0,4405.0,27.24,551.5,320400,325000,20.74,3300,11.93,65.9,17.0,7.58,006c,0.0,645.43,630.78,648.68,1.0,0.0,001b,000000ff,07ffffff
1,04/05/2021 29:29:46,0.0,0.0,0,550.0,0,8,-29,0.0,15,3.607843,0.0,9.90,9.32,913.0,924.0,64.6,55.3,53.2,4391.0,27.36,550.0,322500,325000,20.76,3190,12.30,66.2,17.0,7.50,006c,0.0,645.52,630.78,648.68,1.0,0.0,0000,000000ff,07ffffff
2,04/05/2021 29:29:47,0.0,0.0,0,550.5,0,8,-45,0.0,14,3.450980,0.0,9.62,9.48,912.0,924.0,64.5,55.3,52.6,4344.0,27.70,550.0,324700,325000,20.50,3290,12.58,66.8,17.0,7.60,006c,0.0,645.59,630.78,648.68,1.0,0.0,0000,000000ff,07ffffff
3,04/05/2021 29:29:48,0.0,0.0,0,550.0,0,8,-29,0.0,15,3.607843,0.0,9.70,9.56,912.0,924.0,64.4,55.4,52.1,4371.0,27.84,550.0,324700,325000,20.31,3330,12.17,66.4,17.0,7.54,006c,0.0,645.68,630.78,648.68,1.0,0.0,0000,000001ff,07ffffff
4,04/05/2021 29:29:49,0.0,0.0,0,550.0,0,8,-45,0.0,14,3.450980,0.0,9.56,9.58,911.0,923.0,64.3,55.4,51.6,4303.0,27.92,550.5,324700,325000,20.33,3340,12.02,64.5,17.0,7.12,006c,0.0,645.68,630.78,648.68,1.0,0.0,ffe5,000000ff,07ffffff


Dealing with Engine Grad (rpm)

In [ ]:
EngineGrad = []

for i in dataset['Engine Grad (rpm)']:
  if i != 0:
    EngineGrad.append(((int(i,16))*0.50)+(0))
  else:
    EngineGrad.append(0)

In [ ]:
dataset.drop('Engine Grad (rpm)',axis=1)
dataset['Engine Grad (rpm)'] = (EngineGrad)

In [ ]:
dataset.head()

,Time,Latitude,Longitude,Altitude,RPM,Driver Demand Torque (%),Engine Load (%),Engine Torque Mode,TPS (%),Percent Load Curret Speed,Fuel Rate (L-Hr),Vehicle Speed,Inj Q Cur (mg-st),Inj Q Tor (mg-st),Boost Pressure (mBar),Atmospheric Pressure (mBar),Coolant Temperature (*C),Oil Temperature (*C),Boost Temperature (*C),Oil Pressure (mBar),Battery Voltage (V),Cam Speed (rpm),Rail Pressure (mBar),Rail Pressure set (mBar),MU PWM (%),MU Vol (mm3-s),Torque Rat,Torque (Nm),TQ Limit Set,Main Injection (mg-st),Pilot Injection (mg-st),Pos 2 Injector (mg-st),EGR Prop (%),EGR Pos D (%),EGR Pos A (%),Clutch Switch,Brake Switch,Engine Grad (rpm),param1,param2
0,04/05/2021 29:29:45,0.0,0.0,0,554.0,0,8,-29,0.0,14,3.450980,0.0,9.58,9.04,914.0,924.0,64.6,55.3,54.0,4405.0,27.24,551.5,320400,325000,20.74,3300,11.93,65.9,17.0,7.58,006c,0.0,645.43,630.78,648.68,1.0,0.0,13.5,000000ff,07ffffff
1,04/05/2021 29:29:46,0.0,0.0,0,550.0,0,8,-29,0.0,15,3.607843,0.0,9.90,9.32,913.0,924.0,64.6,55.3,53.2,4391.0,27.36,550.0,322500,325000,20.76,3190,12.30,66.2,17.0,7.50,006c,0.0,645.52,630.78,648.68,1.0,0.0,0.0,000000ff,07ffffff
2,04/05/2021 29:29:47,0.0,0.0,0,550.5,0,8,-45,0.0,14,3.450980,0.0,9.62,9.48,912.0,924.0,64.5,55.3,52.6,4344.0,27.70,550.0,324700,325000,20.50,3290,12.58,66.8,17.0,7.60,006c,0.0,645.59,630.78,648.68,1.0,0.0,0.0,000000ff,07ffffff
3,04/05/2021 29:29:48,0.0,0.0,0,550.0,0,8,-29,0.0,15,3.607843,0.0,9.70,9.56,912.0,924.0,64.4,55.4,52.1,4371.0,27.84,550.0,324700,325000,20.31,3330,12.17,66.4,17.0,7.54,006c,0.0,645.68,630.78,648.68,1.0,0.0,0.0,000001ff,07ffffff
4,04/05/2021 29:29:49,0.0,0.0,0,550.0,0,8,-45,0.0,14,3.450980,0.0,9.56,9.58,911.0,923.0,64.3,55.4,51.6,4303.0,27.92,550.5,324700,325000,20.33,3340,12.02,64.5,17.0,7.12,006c,0.0,645.68,630.78,648.68,1.0,0.0,32754.5,000000ff,07ffffff


################################################################################